TODO :
it is not clear which SAE is best, as they all have pretty similar performance, some being worse on some metrics but better on others.
We chose BatchTopK as they seem to be generally good across all metrics and align with assumptions on the atoms, while MP, though about as good across al metrics, and significantly better in sparse reconstruction, has an encoder that does not align with how atoms are used for subsequent tasks (cosim comparison, linear operations, etc).

Changement de plan : ne pas changer de plan. Batchtopk c'est pourris. La compétition dans topk fait que le dict ne peut tout simplement pas apprendre plusieurs atomes géométriquement proches, faute de quoi ils seraient tous actifs en même temps. Donc si t'as un biais pour le texte et un pour l'image, qui ont une cosim non nulle, le topk sera obligé d'apprendre un terme commun et deux termes quasi orthogonaux. Du coup ça règle la question des features bimodal-divergent, elles ne peuvent juste plus exister dans un batchtopk. C'est stupide.

Du coup, changement de plan, dire "nos assumptions sont IsoE, encoder assumption, linear represenation, superposition, ...", puis "on sépare les features en trois catégories : bimodal aligned, bimodal divergent, modality specific" -> les deux derniers sont indistinguables dans un SAE normal mais pas dans un crosscodeur. Cependant, encodeur assumption => pas de bimodal divergent. Cependant, s'il y en a, on veut les trouver quand même, et un batchtopk ne peut pas. Donc on fait MP, parce que de toute façon, c'est pas plus non linéaire que BatchTopK. Il y a que vanilla ReLU SAE qui colle à l'assumption de linéarité mais personne ne les utilises parce que c'est pourri.

We find that modality gap useless, bimodal <3 <3 <3, but still, unimodal has a little bit of information.
This suggests a violation of our core assumptions, either the IsoE assumption is false, or CLIP violates the encoder assumption (give it a cool name, the assumption that CLIP's towers reverse the DGP that went from c to instances of image and texts, and so that SAEs are only meant for disentangling these representations, not actually reverse the DGP).

# Multimodal Feature Analysis

### 1. Model : CLIP (default)
CLIP (Contrastive Language-Image Pretraining) is a model developed by OpenAI that learns to associate images and text. It can be used for various tasks such as zero-shot classification, image generation, and more. CLIP uses a transformer architecture to process both images and text, allowing it to learn rich multimodal representations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import rfft, rfftfreq

# Parameters
fs = 44100
duration = 1.0
f0 = 440  # Hz

t = np.linspace(0, duration, int(fs*duration), endpoint=False)

# ADSR envelope
attack, decay, sustain, release = 0.1, 0.1, 0.7, 0.2
sustain_time = duration - (attack + decay + release)

env = np.zeros_like(t)
A = int(attack*fs)
D = int(decay*fs)
S = int(sustain_time*fs)
R = int(release*fs)

env[:A] = np.linspace(0, 1, A, endpoint=False)
env[A:A+D] = np.linspace(1, sustain, D, endpoint=False)
env[A+D:A+D+S] = sustain
env[A+D+S:] = np.linspace(sustain, 0, R, endpoint=False)

# Messy violin-like signal: harmonics + vibrato + slight noise
vibrato = 5*np.sin(2*np.pi*6*t)  # 6 Hz vibrato, ±5 Hz
phase = 2*np.pi*(f0*t + np.cumsum(vibrato)/fs)

signal = env * (
    np.sin(phase) +
    0.5*np.sin(2*phase) +
    0.3*np.sin(3*phase) +
    0.2*np.sin(4*phase)
)

# add small random noise for realism
signal += 0.02*np.random.randn(len(t))

# Plot time-domain waveform
plt.figure(figsize=(10,4))
plt.plot(t, signal, lw=0.7)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Violin-like note (time domain, messy waveform)")
plt.grid(True)
plt.show()

# Frequency spectrum
N = len(signal)
yf = np.abs(rfft(signal)) / N
xf = rfftfreq(N, 1/fs)

plt.figure(figsize=(10,4))
plt.stem(xf, yf, basefmt=" ")
plt.xlim(0, 5000)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Amplitude")
plt.title("Violin-like note (frequency domain)")
plt.grid(True)
plt.show()


In [ ]:
import torch
import numpy as np

import overcomplete

from src import losses, train, metrics
from src.utils import (
    get_coco, get_laion, get_imagenet,
    _train,
    measure_everything, measure_bridge, get_bridge_norms,
    get_rho,
)
from src.utils import (
    plot_energy, plot_energy_cdf, plot_E_vs_E_B,
    plot_modality_energy, mu_vs_mu_B,
    plot_bridges,
)

device = "cuda:1" if torch.cuda.is_available() else "cpu"

use_model = "clip"  # openclip or siglip

if use_model == "clip":
    model_name = "openai/clip-vit-base-patch32"
    d_model = 512
    beta = 4e-4
elif use_model == "clip-L":
    model_name = "openai/clip-vit-large-patch14"
    d_model = 768
    beta = 1e-4
elif use_model == "openclip":
    model_name="laion/CLIP-ViT-B-32-laion2B-s34B-b79K"
    d_model = 512
    beta = 4e-4
elif use_model == "openclip-L":
    model_name="laion/CLIP-ViT-L-14-laion2B-s32B-b82K"
    d_model = 768
    beta = 2e-4
elif use_model == "siglip":
    model_name = "google/siglip-base-patch16-224"
    d_model = 768
    beta = 1e-4
elif use_model == "siglip2":
    model_name = "google/siglip2-base-patch16-224"
    d_model = 768
    beta = 7e-5
else:
    raise NotImplementedError(f"Model {use_model} not recognized.")

laion_loader = get_laion(device=device, model_name=model_name)
# coco_loader = get_coco(device=device)
# imagenet_test_loader, class_embeddings = get_imagenet(batch_size=512, model_name=model_name, device=device)
# imagenet_train_loader, class_embeddings = get_imagenet(batch_size=512, model_name=model_name, device=device, split="train")

train_loader = laion_loader
modality_threshold = 0.05

betas = torch.logspace(-5, -2, 100, device=device)  # beta values to try
l2s, r2s, Es_in_degenerate, Es_in_bimodal, Es_tot = [], [], [], [], []

# from tqdm import tqdm

# def wtf(beta):
#     ASAE, ASAE_name, _ = _train(beta=beta, train_loader=train_loader, model_name=model_name, d_model=d_model, UseTOPK=False, device=device) #, force_retrain=True, save_quand_meme=True) # Aligned SAE
#     ASAE.metrics = measure_everything(ASAE, train_loader, device, return_sqr=True)
#     l2, r2 = metrics.l2_r2(ASAE, train_loader)
    
#     E = ASAE.metrics.E
#     f = ASAE.metrics.f
#     mu = ASAE.metrics.mu
#     degenerate_mask = (f > 0.9)
#     bimodal_mask = (mu > modality_threshold) & (mu < (1 - modality_threshold))
    
#     E_in_degenerate = E[degenerate_mask].sum().item()
#     E_in_bimodal = E[bimodal_mask & ~degenerate_mask].sum().item()
#     E_tot = E.sum().item()

#     return l2, r2, E_in_degenerate, E_in_bimodal, E_tot

# for beta in tqdm(betas):
#     l2, r2, E_in_degenerate, E_in_bimodal, E_tot = wtf(beta.item())

#     print(f"Beta: {beta:.6f}, L2: {l2:.4f}, R2: {r2:.4f}, E_in_degenerate: {E_in_degenerate:.4f}, E_in_bimodal: {E_in_bimodal:.4f}, E_tot: {E_tot:.4f}")
#     l2s.append(l2)
#     r2s.append(r2.item())
#     Es_in_degenerate.append(E_in_degenerate / E_tot)
#     Es_in_bimodal.append(E_in_bimodal / E_tot)
#     Es_tot.append(E_tot)
    

In [ ]:
r2s = [0.8471270799636841, 0.8477537035942078, 0.8474643230438232, 0.8472068905830383, 0.8475709557533264, 0.847588837146759, 0.8474730849266052, 0.847104012966156, 0.8470807671546936, 0.8477079272270203, 0.8470604419708252, 0.8473597764968872, 0.8469316363334656, 0.8480101823806763, 0.8480943441390991, 0.8476691246032715, 0.8475169539451599, 0.8472616672515869, 0.8474118709564209, 0.8473469018936157, 0.8474711775779724, 0.8469681739807129, 0.8466691374778748, 0.847098171710968, 0.847135603427887, 0.8471106290817261, 0.8468508720397949, 0.8469724059104919, 0.8464853167533875, 0.8460384011268616, 0.8462086915969849, 0.8461043238639832, 0.845971941947937, 0.8457302451133728, 0.8456987142562866, 0.8448388576507568, 0.8443150520324707, 0.8444170951843262, 0.8386593461036682, 0.8424999713897705, 0.841633141040802, 0.8407778739929199, 0.8404752612113953, 0.8403682112693787, 0.8386974334716797, 0.8353956341743469, 0.8361472487449646, 0.8363955616950989, 0.8357362151145935, 0.8341480493545532, 0.8323978781700134, 0.8298661708831787, 0.821643054485321, 0.8270752429962158, 0.8242298364639282, 0.8244916200637817, 0.8241608738899231, 0.8218496441841125, 0.8202092051506042, 0.8213890790939331, 0.8169975280761719, 0.80110764503479, 0.779854953289032, 0.7974963188171387, 0.7336429357528687, 0.7288471460342407, 0.7546337842941284, 0.7429816722869873, 0.7112916707992554, 0.6983168125152588, 0.6960105895996094, 0.6613096594810486, 0.641830563545227, 0.6298071146011353, 0.5898458361625671, 0.5714426636695862, 0.5319939255714417, 0.35821810364723206, 0.345608651638031, 0.34735584259033203, 0.32399606704711914, 0.3207992911338806, 0.29147595167160034, 0.3034614622592926, 0.28855839371681213, 0.28426653146743774, 0.290071040391922, 0.29245632886886597, 0.2676733434200287, 0.28811293840408325, 0.24682894349098206, 0.2725927233695984, 0.26762112975120544, 0.25793737173080444, 0.27988436818122864, 0.2779000997543335, 0.31128180027008057, 0.28164151310920715, 0.29129403829574585, 0.2870906889438629]
l2s = [0.15286938533330408, 0.15224318892046665, 0.15253203936695034, 0.15278984544552757, 0.15242620526723433, 0.1524078126150247, 0.15252439731400824, 0.15289238146727954, 0.1529156966076579, 0.1522889219692905, 0.15293670671042744, 0.1526369882576147, 0.15306491494331298, 0.15198655828123458, 0.15190236236477045, 0.152327117091673, 0.15247982050819203, 0.15273450172941047, 0.152584720694547, 0.1526492706918377, 0.15252555536727166, 0.15302784066025893, 0.15332804651875193, 0.1528991094875631, 0.15286142426191746, 0.15288580638952545, 0.15314667915042374, 0.15302405330213692, 0.15351139228122773, 0.1539579382402792, 0.15378751139287158, 0.1538923883643353, 0.15402468997419755, 0.15426656561065447, 0.15429803579206133, 0.15515817045314526, 0.15568217853174812, 0.15557940669909517, 0.1613363983837233, 0.15749622268929256, 0.15836381350557113, 0.1592189510612954, 0.15952131037690798, 0.15962883563950855, 0.1612990745629988, 0.16459954433717958, 0.16384875097578014, 0.16360057636303715, 0.16426033429172124, 0.16584767118407762, 0.16759936518039112, 0.17012962216669875, 0.17835134841991238, 0.17292023112753613, 0.175764979279299, 0.17550300343376118, 0.17583434381370777, 0.17814412718001282, 0.17978545812306124, 0.1786057345600743, 0.18299688840798678, 0.1988869497796315, 0.220139156055733, 0.2024978810752018, 0.26635042231032985, 0.27114674262522676, 0.24535959933475174, 0.2570118544056171, 0.2887034458116922, 0.30168051423468617, 0.3039869844475971, 0.3386869344872608, 0.3581656570742193, 0.3701886712946863, 0.4101486517022645, 0.4285513778822374, 0.46799837037201464, 0.6417855868561854, 0.6543897325554965, 0.652643958064784, 0.6760020798584029, 0.6792016564139308, 0.7085371184326047, 0.6965386608494725, 0.7114448601951925, 0.7157360116181362, 0.7099310279323339, 0.7075484033899196, 0.7323288948929975, 0.711890027323364, 0.7531706445125991, 0.7274143595083139, 0.7323849247549222, 0.7420714045858573, 0.7201271717276873, 0.7221099101573107, 0.6887323904118824, 0.718367292821125, 0.7087201779184498, 0.7129216845604077]
Es_in_degenerate = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25959705802115446, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.27226849322022256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15741910566189574, 0.2513232412854947, 0.0, 0.24499231098397647, 0.23483869994328177, 0.0, 0.0, 0.0, 0.0, 0.4745973940360938, 0.6573640649700063, 0.4808152547344781, 0.8224089447025716, 0.8333885152342984, 0.7575605223379728, 0.7857557576704108, 0.8593726591522545, 0.8694006327483957, 0.8712143595763083, 0.8789431222039618, 0.8829776980130783, 0.8865836340282149, 0.8950623881093239, 0.8991387111450867, 0.9038383389560609, 0.9341428329999175, 0.9378026423642712, 0.9390313287359725, 0.9451222755770702, 0.9495898515273906, 0.9960121443433746, 0.9563202772313331, 0.966600622864906, 0.9695309212471561, 0.9644047419005354, 0.9678104158936472, 0.987660191159391, 0.973167356758933, 0.9885628612833419, 0.9928619036340316, 0.9939286809293985, 0.9975405184027062, 0.9977101456811727, 0.9984801046417172, 0.9961143289998017, 0.9990889805874419, 0.998109468097601, 0.998993555480274]
Es_in_bimodal = [0.28056837325001266, 0.2791806923769942, 0.2830089257627494, 0.28124897603186805, 0.28090208147506174, 0.2831744600809655, 0.29006049094491376, 0.2872127887582236, 0.2791614607134614, 0.28550153695829344, 0.2857420326548741, 0.28828113094444807, 0.2828630079638485, 0.2830982384762928, 0.28645304675943806, 0.2963123448047671, 0.2936925080550728, 0.2910731141079016, 0.2962508180576517, 0.2956354953224143, 0.2905085813243365, 0.29743937156472156, 0.3036051407390845, 0.30426776258428073, 0.3025420033923383, 0.3045520605464526, 0.31258648282844725, 0.30936505353149174, 0.31330395516068793, 0.3212161871398127, 0.32254171311857155, 0.3267352317949248, 0.33026604735200177, 0.33258094430182317, 0.3377788189351328, 0.3511810018810298, 0.3541530433717741, 0.3534827067821889, 0.30708496601375834, 0.40691200335984995, 0.4071046018942464, 0.43606577045698025, 0.42003259319184116, 0.4364762862567092, 0.46682953957621054, 0.34520978613793335, 0.4343756997258249, 0.4477808948286244, 0.48154161415041713, 0.4743911555398545, 0.46275439338771035, 0.49307684925826617, 0.45602931594685675, 0.4313914253054964, 0.6026343865812198, 0.43258769501214783, 0.45629202591125007, 0.6707088137460079, 0.6948313411441596, 0.6923687647341309, 0.7177604380037602, 0.30697514145311733, 0.20257364907140765, 0.3156055419852461, 0.06047494606413658, 0.06200362262253669, 0.15029854962603273, 0.13337246554581247, 0.04524669693037746, 0.026475075206497205, 0.032929738705043954, 0.0240644653312398, 0.027017774753220848, 0.025496257295700785, 0.02333846665964243, 0.02147445082258719, 0.022709746695593055, 0.054535002770823425, 0.04519382770106961, 0.05467140107887884, 0.05298883613142839, 0.048926941033538396, 0.003968089667603372, 0.04314398893102642, 0.0333860652592096, 0.030458024129735976, 0.035587377549412345, 0.03217252623500613, 0.012328079214585856, 0.026826435673041743, 0.01142290903133892, 0.007134234362986226, 0.006066983936903304, 0.002454568488430711, 0.002275146540028688, 0.001515947317531605, 0.003868772279813372, 0.0008781744283922075, 0.0018853443691432541, 0.001004370291531495]
Es_tot = [0.8477544784545898, 0.848163366317749, 0.847916305065155, 0.8476754426956177, 0.8479911088943481, 0.8482930064201355, 0.8480300903320312, 0.8476519584655762, 0.8475362062454224, 0.8481579422950745, 0.8475993871688843, 0.847968578338623, 0.8474289178848267, 0.848482608795166, 0.848540186882019, 0.8481056690216064, 0.8483318090438843, 0.8478139638900757, 0.8479219675064087, 0.8478422164916992, 0.8478667736053467, 0.8474538326263428, 0.8471779227256775, 0.8476243615150452, 0.8475947380065918, 0.8475282788276672, 0.847251832485199, 0.847458004951477, 0.8470689058303833, 0.8465664386749268, 0.8469842672348022, 0.8476683497428894, 0.8466296195983887, 0.8462467193603516, 0.8475314974784851, 0.8454155921936035, 0.8449605703353882, 0.8451530933380127, 0.873070478439331, 0.8458560705184937, 0.8426194787025452, 0.841567873954773, 0.8414056897163391, 0.841474175453186, 0.8410202860832214, 0.8782918453216553, 0.8374539613723755, 0.8380166292190552, 0.8402600288391113, 0.836012065410614, 0.8348323106765747, 0.8360180854797363, 0.8840599060058594, 0.8779496550559998, 0.8719735145568848, 0.8793391585350037, 0.8710829615592957, 0.8716825246810913, 0.8615025281906128, 0.8709375262260437, 0.8850370049476624, 1.0625784397125244, 1.443810224533081, 1.1098324060440063, 1.8603291511535645, 1.9219441413879395, 1.8525278568267822, 2.0379889011383057, 2.065673351287842, 2.1329803466796875, 2.1310999393463135, 2.1144330501556396, 2.095106363296509, 2.0991156101226807, 2.0523765087127686, 2.0400235652923584, 1.9595680236816406, 1.5165627002716064, 1.5186126232147217, 1.5108684301376343, 1.4633742570877075, 1.4669060707092285, 1.5931816101074219, 1.4436969757080078, 1.4344964027404785, 1.4328787326812744, 1.438012957572937, 1.4675893783569336, 1.476189374923706, 1.4798977375030518, 1.0706192255020142, 1.5370910167694092, 1.527235984802246, 1.5297387838363647, 1.5772027969360352, 1.1928024291992188, 1.5661733150482178, 1.1944228410720825, 1.504199743270874, 1.499762773513794]

In [ ]:
print(r2s)
print(l2s)
print(Es_in_degenerate)
print(Es_in_bimodal)
print(Es_tot)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

nthbeta = 60
threshold = betas[nthbeta].item()

plt.plot(betas.cpu().numpy(), l2s, label='L2')
plt.plot(betas.cpu().numpy(), r2s, label='R2')
plt.axvline(threshold, color='red', linestyle='--', label=f'Beta = {threshold:.1e}')
plt.xscale('log')
# plt.xlabel('Beta')
# plt.ylabel('Metric Value')
# plt.title('L2 and R2 vs Beta')
plt.legend()
plt.tight_layout()
plt.savefig("l2_r2_vs_beta.png")
plt.figure()

####
# V1
####

# stackplot, normalize Es_in_degenerate and Es_in_bimodal
Es_in_degenerate_array = np.array(Es_in_degenerate)
Es_in_bimodal_array = np.array(Es_in_bimodal)
plt.stackplot(betas.cpu().numpy(), Es_in_bimodal_array, Es_in_degenerate_array, labels=['Es_in_bimodal', 'E_in_degenerate'])
plt.axvline(threshold, color='red', linestyle='--', label=f'Beta = {threshold:.1e}')
plt.xscale('log')
# plt.xlabel('Beta')
# plt.ylabel('Energy Fraction')
# plt.title('Energy in Degenerate and Bimodal Regions vs Beta')
plt.legend()
plt.tight_layout()
plt.savefig("energy_vs_beta.png")
plt.figure()

####
# V2
####
"""
def smooth(y, window=5):
    return np.convolve(y, np.ones(window)/window, mode='same')

Es_in_degenerate_smooth = smooth(Es_in_degenerate_array, window=5)
Es_in_bimodal_smooth    = smooth(Es_in_bimodal_array, window=5)

plt.stackplot(betas.cpu().numpy(), Es_in_bimodal_smooth, Es_in_degenerate_smooth, labels=['E_in_bimodal', 'E_in_degenerate'])
plt.xscale('log')
plt.legend()
plt.tight_layout()
plt.savefig("energy_vs_beta_smooth.png")
plt.figure()

####
# V3
####

plt.stackplot(betas.cpu().numpy(), Es_in_bimodal_array, Es_in_degenerate_array,
              labels=['bimodal', 'degenerate'], alpha=0.2)

# Smoothed curves in bold
plt.plot(betas.cpu().numpy(), Es_in_bimodal_smooth, color='C0', lw=2)
plt.plot(betas.cpu().numpy(), Es_in_degenerate_smooth + Es_in_bimodal_smooth, color='C1', lw=2)

plt.xscale('log')
plt.legend()
plt.tight_layout()
plt.savefig("energy_vs_beta_overlay.png")
plt.figure()"""

In [ ]:
from matplotlib import pyplot as plt
import torch
import numpy as np
from tqdm import tqdm

# siglip :
img_highE = [
    366, 3160, 5780, 5626, 3146, 4824, 4712, 4042, 6143
]
img_lowE = [
    5523, 4028, 6017, 4064, 274, 1756, 3415
]
txt_highE = [
    1112, 3855, 3122, 5581,
]
txt_lowE = [
    5353, 6065, 4036, 3983, 2412, 5831, 697, 3138, 5895,
]
bi_idxs_highE = [
    1195, 2465, 991, 273, 2739, 1793, 4774, 5806, 2811, 4477, 858, 5899, 5163
]
bi_idxs_lowE = [
    3554, 352, 48, 4717, 3597, 5425, 3507, 5602, 2830, 1173, 2387, 4338, 4933, 229, 5797, 4182, 3525
]

def plot_app_pacc(D, I, T, Z_I, Z_T):
    print(I.shape)
    
    for i, idx in enumerate(bi_idxs_highE + bi_idxs_lowE):
        D_i = D[idx].unsqueeze(0)  # Shape (1, d_model)
        Ii = I @ D_i.T  # Shape (n_samples, 1)
        Ti = T @ D_i.T  # Shape (n_samples, 1)
        # concatenate to 2*n_samples, shape (2*n_samples, 1)
        Ai = torch.cat([Ii, Ti], dim=0).cpu().detach()
        Z_I_i, Z_T_i = Z_I[:, i], Z_T[:, i]  # Shape (n_samples,)
        active_idxs = (Z_I_i != 0) | (Z_T_i != 0)  # Shape (n_samples,)
        print(f"Feature {i} - active samples: {active_idxs.sum()}/{len(active_idxs)}")
         
        bins = np.linspace(Ai.min(), Ai.max(), 100)
        bin_centers = 0.5 * (bins[:-1] + bins[1:])
        digitized = np.digitize(Ai, bins) - 1 # shape (n_samples,) : contains the bin index for each sample
        weights = np.ones_like(Ai) / len(Ai)  # Uniform weights for density plot
        
        plt.figure(figsize=(10, 6))
        for b in range(len(bins) - 1):
            b_idx = np.where(digitized == b)[0]
            if len(b_idx) == 0:
                continue
            
            bottom = 0
            
            for zgzfeg in [0, 1]:  # 0 for active, 1 for inactive
                for aerzg in [0, 1]:  # 0 for I, 1 for T
                    if aerzg == 0 and zgzfeg == 0:
                        continue
                    elif aerzg == 1 and zgzfeg == 0:
                        continue
                    elif aerzg == 0 and zgzfeg == 1:
                        color = np.array([128/255, 230/255, 255/255]) # Light blue
                        retained_indices = b_idx[b_idx < len(I)]
                        retained_indices = retained_indices[~active_idxs[retained_indices]]
                    elif aerzg == 1 and zgzfeg == 1:
                        color = np.array([255/255, 205/255, 102/255])  # Light orange
                        retained_indices = b_idx[b_idx >= len(I)] - len(I)
                        retained_indices = retained_indices[~active_idxs[retained_indices]]
                    height = weights[retained_indices].sum()
                    plt.bar(bin_centers[b], height, width=(bins[1] - bins[0]), bottom=bottom, color=color, edgecolor='none')
                    bottom += height
        plt.title(f"Distribution of {['High', 'Low'][i >= len(bi_idxs_highE)]} Energy Feature {idx}'s Activations")
        plt.xlabel("Activation Value")
        plt.ylabel("Density")
        # plt.yscale('log')
        plt.show()
        
        max_height = 0
        for b in range(len(bins) - 1):
            b_idx = np.where(digitized == b)[0]
            bin_height = weights[b_idx].sum()
            if bin_height > max_height:
                max_height = bin_height
        print(f"Max height: {max_height}")
        max_height *= 0.1
        
        plt.figure(figsize=(10, 6))
        for b in range(len(bins) - 1):
            b_idx = np.where(digitized == b)[0]
            if len(b_idx) == 0:
                continue
            
            bottom = 0
            
            for zgzfeg in [0, 1]:  # 0 for active, 1 for inactive
                for aerzg in [0, 1]:  # 0 for I, 1 for T
                    if aerzg == 0 and zgzfeg == 0:
                        color = np.array([0/255, 145/255, 181/255])  # Dark blue
                        retained_indices = b_idx[b_idx < len(I)]
                        retained_indices = retained_indices[active_idxs[retained_indices]]
                    elif aerzg == 1 and zgzfeg == 0:
                        color = np.array([208/255, 140/255, 0/255])  # Dark orange
                        retained_indices = b_idx[b_idx >= len(I)] - len(I)
                        retained_indices = retained_indices[active_idxs[retained_indices]]
                    elif aerzg == 0 and zgzfeg == 1:
                        color = np.array([128/255, 230/255, 255/255]) # Light blue
                        retained_indices = b_idx[b_idx < len(I)]
                        retained_indices = retained_indices[~active_idxs[retained_indices]]
                    elif aerzg == 1 and zgzfeg == 1:
                        color = np.array([255/255, 205/255, 102/255])  # Light orange
                        retained_indices = b_idx[b_idx >= len(I)] - len(I)
                        retained_indices = retained_indices[~active_idxs[retained_indices]]
                    height = weights[retained_indices].sum()
                    if height+bottom > max_height: height = max_height - bottom
                    plt.bar(bin_centers[b], height, width=(bins[1] - bins[0]), bottom=bottom, color=color, edgecolor='none')
                    bottom += height
                    if bottom >= max_height:
                        bottom = max_height
        plt.title(f"Distribution of {['High', 'Low'][i >= len(bi_idxs_highE)]} Energy Feature {idx}'s Activations")
        plt.xlabel("Activation Value")
        plt.ylabel("Density")
        # plt.yscale('log')
        plt.show()
        
        plt.figure(figsize=(10, 6))
        for b in range(len(bins) - 1):
            b_idx = np.where(digitized == b)[0]
            if len(b_idx) == 0:
                continue
            
            bottom = 0
            
            for zgzfeg in [0, 1]:  # 0 for active, 1 for inactive
                for aerzg in [0, 1]:  # 0 for I, 1 for T
                    if aerzg == 0 and zgzfeg == 0:
                        color = np.array([0/255, 145/255, 181/255])  # Dark blue
                        retained_indices = b_idx[b_idx < len(I)]
                        retained_indices = retained_indices[active_idxs[retained_indices]]
                    elif aerzg == 1 and zgzfeg == 0:
                        color = np.array([208/255, 140/255, 0/255])  # Dark orange
                        retained_indices = b_idx[b_idx >= len(I)] - len(I)
                        retained_indices = retained_indices[active_idxs[retained_indices]]
                    elif aerzg == 0 and zgzfeg == 1:
                        color = np.array([128/255, 230/255, 255/255]) # Light blue
                        retained_indices = b_idx[b_idx < len(I)]
                        retained_indices = retained_indices[~active_idxs[retained_indices]]
                    elif aerzg == 1 and zgzfeg == 1:
                        color = np.array([255/255, 205/255, 102/255])  # Light orange
                        retained_indices = b_idx[b_idx >= len(I)] - len(I)
                        retained_indices = retained_indices[~active_idxs[retained_indices]]
                    height = weights[retained_indices].sum()
                    plt.bar(bin_centers[b], height, width=(bins[1] - bins[0]), bottom=bottom, color=color, edgecolor='none')
                    bottom += height
        plt.title(f"Distribution of {['High', 'Low'][i >= len(bi_idxs_highE)]} Energy Feature {idx}'s Activations")
        plt.xlabel("Activation Value")
        plt.ylabel("Density")
        # plt.yscale('log')
        plt.show()

# for all features in bi_idxs_highE:

# Make the following plots :
# 1. Histogram of A * D_i, the dot products between the embeddings and the feature.
# 2. Same, but restrict A to image-caption pairs where Z_i is not zero.

# Get D_i, Z_i and A_i.
D = ASAE.dictionary._fused_dictionary.cpu().detach()
I, T = train_loader.dataset.tensors[0].cpu().detach(), train_loader.dataset.tensors[1].cpu().detach()

@torch.no_grad()
def get_Zi(i):
    Z_I_i = []
    Z_T_i = []
    for img, txt in tqdm(train_loader, desc=f"Computing Z_{i}"):
        img = img.to(device)
        txt = txt.to(device)
        _, z_img, _ = ASAE(img) # Shape (b, K)
        _, z_txt, _ = ASAE(txt) # Shape (b, K)
        
        Z_I_i.append(z_img[:, i].cpu().numpy())  # Shape (b,)
        Z_T_i.append(z_txt[:, i].cpu().numpy()) # Shape (b,)
    Z_I_i = np.concatenate(Z_I_i)  # Shape (n_samples,)
    Z_T_i = np.concatenate(Z_T_i) # Shape (n_samples,)
    return Z_I_i, Z_T_i

@torch.no_grad()
def get_Zis(idxs):
    Z_I_i = []
    Z_T_i = []
    for img, txt in tqdm(train_loader, desc=f"Computing Z"):
        img = img.to(device)
        txt = txt.to(device)
        _, z_img, _ = ASAE(img) # Shape (b, K)
        _, z_txt, _ = ASAE(txt) # Shape (b, K)
        
        Z_I_i.append(z_img[:, idxs].cpu().numpy())  # Shape (b, k)
        Z_T_i.append(z_txt[:, idxs].cpu().numpy()) # Shape (b, k)
    Z_I_i = np.concatenate(Z_I_i)  # Shape (n_samples, k)
    Z_T_i = np.concatenate(Z_T_i) # Shape (n_samples, k)
    return Z_I_i, Z_T_i

from matplotlib import pyplot as plt
from seaborn import histplot

# Z_I, Z_T = get_Zis(bi_idxs_highE + bi_idxs_lowE)  # Shape (n_samples, k)
print(f"Z_I shape: {Z_I.shape}, Z_T shape: {Z_T.shape}")

plot_app_pacc(D, I, T, Z_I, Z_T)

"""
# for i, idx in enumerate(bi_idxs_highE + bi_idxs_lowE):
    # D_i = D[idx].unsqueeze(0)  # Shape (1, d_model)
    # Ii = I @ D_i.T  # Shape (n_samples, 1)
    # Ti = T @ D_i.T  # Shape (n_samples, 1)
    # #Z_I_i, Z_T_i = get_Zi(i)  # Shape (n_samples,)
    # Z_I_i, Z_T_i = Z_I[:, i], Z_T[:, i]  # Shape (n_samples,)
    # active_idxs = (Z_I_i != 0) | (Z_T_i != 0)  # Shape (n_samples,)
    # print(f"Feature {i} - active samples: {active_idxs.sum()}/{len(active_idxs)}")
    
    # plt.figure(figsize=(12, 6))
    # plt.subplot(1, 2, 1)
    # histplot(Ii, bins=100, kde=True, label="Image", stat="density")
    # histplot(Ti, bins=100, kde=True, label="Text", stat="density")
    # plt.title(f"Dot products A * D_{i} - All samples")
    # plt.xlabel("Dot product value")
    # plt.ylabel("Density")
    # plt.legend()
    # plt.subplot(1, 2, 2)
    # histplot(Ii[active_idxs], bins=100, kde=True, label="Image", stat="density")
    # histplot(Ti[active_idxs], bins=100, kde=True, label="Text", stat="density")
    # plt.title(f"Dot products A * D_{i} - Active samples (Z_{i} != 0)")
    # plt.xlabel("Dot product value")
    # plt.ylabel("Density")
    # plt.legend()
    # plt.tight_layout()
    # plt.show()
    # break
"""

In [ ]:
import re
import numpy as np

def parse_latex_table(table_str):
    rows = []
    for line in table_str.strip().splitlines():
        line = line.strip()
        if not line or not line.endswith(r"\\"):
            continue
        parts = [x.strip() for x in line[:-2].split("&")]
        label = parts[0]
        try:
            values = [float(x) for x in parts[1:] if x.replace(".", "", 1).isdigit()]
        except ValueError:
            values = []
        rows.append((label, values))
    return rows

def compute_differences(rows):
    ref_values = np.array(rows[0][1])
    new_rows = []
    for label, values in rows:
        values = np.array(values)
        if len(values) != len(ref_values):
            delta = "..."
        else:
            delta = round(np.mean(np.abs(values - ref_values)), 3)
        new_row = f"{label} & " + " & ".join(map(str, values)) + f" & {delta} \\\\"
        new_rows.append(new_row)
    return new_rows


In [ ]:
latex_table =r"""\quad $(\widehat{I}, \widehat{T})$ & 0.882 & 0.909 & 0.952 & 0.964 & 0.907 & 0.307 & ... \\
        \quad $(I_{\mathrm{center}}, T_{\mathrm{center}})$ & 0.898 & 0.917 & 0.951 & 0.966 & 0.921 & 0.330 & ... \\
        \quad $(\widetilde{I}, \widetilde{T})$ & 0.760 & 0.891 & 0.933 & 0.964 & 0.901 & 0.313 & ... \\
        \quad $(I_{\mathrm{rand}}, I_{\mathrm{rand}})$ & 0.745 & 0.830 & 0.924 & 0.943 & 0.856 & 0.251 & ... \\
        \quad $(I_{\mathrm{shift}}, I_{\mathrm{shift}})$ & 0.704 & 0.798 & 0.913 & 0.935 & 0.827 & 0.238 & ... \\
"""

# Extract numeric rows
rows = []
for line in latex_table.strip().splitlines():
    numbers = re.findall(r"\d+\.\d+", line)
    rows.append([float(x) for x in numbers])

print(rows)
# Compute and print average absolute differences
ref = np.array(rows[0])
for i, row in enumerate(rows):
    diff = (np.array(row) - ref).mean()
    print(f"Row {i}: Δ = {diff:.3f}")

In [ ]:
import re
import numpy as np

latex_table = r"""\\quad MSE ($\\downarrow$)     & 0.141 $\\mid$ 0.163 & 0.207 $\\mid$ 0.213 & 0.246 $\\mid$ 0.257 & 0.244 $\\mid$ 0.253 & 0.212 $\\mid$ 0.214 & 0.115 $\\mid$ 0.115 \\
        \\quad $R^2$ ($\\uparrow$)   & 0.859 $\\mid$ 0.837 & 0.793 $\\mid$ 0.787 & 0.754 $\\mid$ 0.742 & 0.755 $\\mid$ 0.747 & 0.788 $\\mid$ 0.784 & 0.884 $\\mid$ 0.885 \\
        \\quad $\\ell_0$ ($\downarrow$)   & 20 $\\mid$ 20 & 20 $\\mid$ 20 & 20 $\\mid$ 20 & 20 $\\mid$ 20 & 20 $\\mid$ 20 & 20 $\\mid$ 20 \\
        \\quad $\\ell_1$ ($\downarrow$)     & 2.941 $\\mid$ 3.092 & 2.986 $\\mid$ 2.996 & 3.155 $\\mid$ 3.164 & 3.137 $\\mid$ 3.151 & 2.996 $\\mid$ 2.950 & 2.726 $\\mid$ 2.474 \\
        \\quad $\\rho$ ($\\uparrow$)   & 0.327 $\\mid$ 4.232 & 1.566 $\\mid$ 4.086 & 4.072 $\\mid$ 16.02 & 8.737 $\\mid$ 16.58 & 1.370 $\\mid$ 2.182 & 0.713 $\\mid$ 1.475 \\
        \\quad $\delta_{\\mathrm{r@1}}$ ($\downarrow$)   & -0.224 $\\mid$ -0.125 & -0.039 $\\mid$ -0.021 & -0.037 $\\mid$ -0.018 & -0.001 $\\mid$ +0.000 & -0.023 $\\mid$ -0.006 & -0.007 $\\mid$ +0.006 \\
        \\quad $\\mathrm{FDA}$ ($\\uparrow$)   & 2.630 $\\mid$ 4.559 & 3.914 $\\mid$ 4.800 & 4.369 $\\mid$ 8.160 & 9.787 $\\mid$ 16.49 & 8.831 $\\mid$ 34.95 & 8.246 $\\mid$ 18.24 \\
        \\quad $p_{\\mathrm{acc}}$ ($\\uparrow$)   & 0.847 $\\mid$ 0.915 & 0.843 $\\mid$ 0.868 & 0.849 $\\mid$ 0.880 & 0.845 $\\mid$ 0.873 & 0.897 $\\mid$ 0.899 & 0.886 $\\mid$ 0.903 \\"""

for line in latex_table.strip().splitlines():
    numbers = re.findall(r"\d+\.\d+", line)
    pairs = []
    for i in range(0, len(numbers), 2):
        if i + 1 < len(numbers):
            pairs.append((numbers[i], numbers[i + 1]))
            
    if not pairs:
        continue
    left_vals = np.array([float(l) for l, _ in pairs])
    right_vals = np.array([float(r) for _, r in pairs])
    mean_left = np.mean(left_vals)
    std_left = np.std(left_vals)
    mean_right = np.mean(right_vals)
    std_right = np.std(right_vals)
    print(f"Left: {mean_left:.3f} +- {std_left:.3f}, Right: {mean_right:.3f} +- {std_right:.3f}")
    mean_diff = np.mean(right_vals - left_vals)
    std_diff = np.std(right_vals - left_vals)
    mean_ratio = np.mean(right_vals / left_vals)
    std_ratio = np.std(right_vals / left_vals)
    print(f"Δ = {mean_diff:.3f} +- {std_diff:.3f}, Ratio = {mean_ratio:.3f} +- {std_ratio:.3f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
u = np.linspace(0, 2*np.pi, 2000)
v = np.linspace(0, np.pi, 1000)
R = 1

# Sphere surface
x = R * np.outer(np.cos(u), np.sin(v))
y = R * np.outer(np.sin(u), np.sin(v))
z = R * np.outer(np.ones_like(u), np.cos(v))

# ax.plot_surface(x, y, z, color='lightblue', alpha=0.3, edgecolor='none')

def draw_circle(axis, fixed_val, ax, linestyle_front='-', linestyle_back=':'):
    t = np.linspace(0, 2*np.pi, 400)
    r = np.sqrt(1 - fixed_val**2)

    # Circle coordinates depending on axis
    if axis == 'x':
        fx = np.full_like(t, fixed_val)
        fy = r * np.cos(t)
        fz = r * np.sin(t)
    elif axis == 'y':
        fx = r * np.cos(t)
        fy = np.full_like(t, fixed_val)
        fz = r * np.sin(t)
    elif axis == 'z':
        fx = r * np.cos(t)
        fy = r * np.sin(t)
        fz = np.full_like(t, fixed_val)

    # Get camera direction from view_init
    elev, azim = np.radians(ax.elev), np.radians(ax.azim)
    cam_dir = np.array([
        np.cos(elev) * np.cos(azim),
        np.cos(elev) * np.sin(azim),
        np.sin(elev)
    ])

    # Points on circle
    pts = np.vstack((fx, fy, fz)).T
    dots = pts @ cam_dir  # dot product with camera direction

    front = dots >= 0
    
    # Helper: plot continuous segments
    def plot_segments(mask, style):
        idx = np.where(mask[:-1] != mask[1:])[0] + 1
        split_fx = np.split(fx, idx)
        split_fy = np.split(fy, idx)
        split_fz = np.split(fz, idx)
        split_mask = np.split(mask, idx)
        for xseg, yseg, zseg, mseg in zip(split_fx, split_fy, split_fz, split_mask):
            if np.all(mseg):  # all front or all back
                ax.plot(xseg, yseg, zseg, 'k' + style, linewidth=0.3)

    plot_segments(front, linestyle_front)
    plot_segments(~front, linestyle_back)


# --- Point cloud ---
n_points = 500
color_I = np.array([128/255, 230/255, 255/255])
color_T = np.array([255/255, 205/255, 102/255])

# Image points :
mean = np.array([0, 0.5, 1])
std_dev = 0.1
points1 = mean + std_dev * np.random.randn(n_points, 3)
mean = np.array([0, -0.5, 1])
points2 = mean + std_dev * np.random.randn(n_points, 3)
I = np.vstack((points1, points2))

# Text points :
mean = np.array([1, 0., 0.5])
points3 = mean + std_dev * np.random.randn(n_points, 3)
mean = np.array([1, -0., -0.5])
points4 = mean + std_dev * np.random.randn(n_points, 3)
T = np.vstack((points3, points4))


def draw_raw(I, T):
    fig = plt.figure(figsize=(10,10), facecolor="#FFE5DF00")
    ax = fig.add_subplot(111, projection='3d')

    ax.view_init(elev=10, azim=-70)
    draw_circle('x', 0, ax=ax)
    draw_circle('y', 0, ax=ax)
    draw_circle('z', 0, ax=ax)

    # Tropics on xy-plane (z = ±2/3 R)
    # draw_circle('z', 2/3*R, ax=ax)
    # draw_circle('z', -2/3*R, ax=ax)
    # draw_circle('y', 2/3*R, ax=ax)
    # draw_circle('y', -2/3*R, ax=ax)
    # draw_circle('x', 2/3*R, ax=ax)
    # draw_circle('x', -2/3*R, ax=ax)

    ax.scatter(I[:, 0], I[:, 1], I[:, 2], c=color_I, s=5, label='Image')
    ax.scatter(T[:, 0], T[:, 1], T[:, 2], c=color_T, s=5, label='Text')

    ax.set_box_aspect([1,1,1])
    ax.axis('off')
    plt.gca().set_facecolor("#FFE5DF00")
    plt.savefig('sphere_with_tropics.png', bbox_inches='tight', dpi=300)
    plt.show()

draw_raw(I, T)
I = I / np.linalg.norm(I, axis=1, keepdims=True)
T = T / np.linalg.norm(T, axis=1, keepdims=True)
draw_raw(I, T)
muI = I.mean(axis=0)
muT = T.mean(axis=0)
I -= muI
T -= muT
draw_raw(I, T)
draw_raw(I / np.linalg.norm(I, axis=1, keepdims=True), T / np.linalg.norm(T, axis=1, keepdims=True))

draw_raw(I + 0.5*(muI + muT), T + 0.5*(muI + muT))
draw_raw((I + 0.5*(muI + muT)) / np.linalg.norm(I + 0.5*(muI + muT), axis=1, keepdims=True), (T + 0.5*(muI + muT)) / np.linalg.norm(T + 0.5*(muI + muT), axis=1, keepdims=True))

r = np.random.randn(3)
r /= np.linalg.norm(r)
draw_raw(I + r, T + r)
draw_raw((I + r) / np.linalg.norm(I + r, axis=1, keepdims=True), (T + r) / np.linalg.norm(T + r, axis=1, keepdims=True))

In [ ]:
I_ = I + muI
T_ = T + muT
muI1 = I_[:n_points].mean(axis=0) * 1
muT1 = T_[:n_points].mean(axis=0) * 1
muI2 = I_[n_points:].mean(axis=0) * 1
muT2 = T_[n_points:].mean(axis=0) * 1
I1 = I_[:n_points] - muI1
T1 = T_[:n_points] - muT1
I2 = I_[n_points:] - muI2
T2 = T_[n_points:] - muT2
I_ = np.zeros_like(I_)
I_[:n_points] = I1
I_[n_points:] = I2
T_ = np.zeros_like(T_)
T_[:n_points] = T1
T_[n_points:] = T2

I_ = I_
T_ = T_

print(I2.shape, T2.shape)

draw_raw(I_, T_)
draw_raw(I_ / np.linalg.norm(I_, axis=1, keepdims=True), T_ / np.linalg.norm(T_, axis=1, keepdims=True))

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

from src.utils import get_coco, get_laion, get_imagenet, get_fashionIQ

In [ ]:
get_fashionIQ(model_name="google/siglip-base-patch16-224")

In [ ]:
get_imagenet()

In [ ]:
get_laion()

In [ ]:
get_coco()

In [ ]:
import os, requests, zipfile, random
import copy
from dataclasses import dataclass, field

import torch
from torch.nn import functional as F
import numpy as np
# from nnsight.models.UnifiedTransformer import UnifiedTransformer
import clip
import nnsight

import overcomplete
from overcomplete.metrics import r2_score
import losses
import train

import datasets

import pandas as pd
from pycocotools.coco import COCO

from timm.data.dataset import ImageDataset
from transformers import CLIPModel, CLIPProcessor, SiglipModel, SiglipProcessor, AutoProcessor, AutoModel

from typing import Callable


from tqdm import tqdm

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import PathPatch, FancyArrowPatch, Circle
from matplotlib.gridspec import GridSpec
from matplotlib.collections import LineCollection
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from matplotlib.colors import LogNorm, LinearSegmentedColormap
import colorsys

import umap
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

from src.utils.data import get_coco, get_laion, get_imagenet_clip_embeddings
import metrics

device = "cuda:1" if torch.cuda.is_available() else "cpu"

## 2. Dataset
COCO (Common Objects in Context) is a large-scale dataset for object detection, segmentation, and captioning. It contains over 100k images, each with ~ 5 captions

LAION 400M is a large-scale dataset of image-text pairs, designed for training and evaluating multimodal models. It contains 400 million image-text pairs, curated using a CLIP-based filtering process. We only use the first 1M of them.

In [ ]:
DATASET = "laion"
batch_size = 512

if DATASET == "coco":
    train_loader = get_coco(batch_size=batch_size, device=device)
elif DATASET == "laion":
    train_loader = get_laion(batch_size=batch_size, device=device)

## 3. Feature Extraction

#### 3.1. SAE training

In [ ]:
SSAE, SSAE_name, _ = __train(beta=0, UseTOPK=False) # Standard SAE
ASAE, ASAE_name, _ = __train(beta=4e-4, UseTOPK=False)#, force_retrain=True, save_quand_meme=True) # Aligned SAE

#### 3.2. Useful Measures

In [ ]:
modality_threshold = 0.05

In [ ]:
SSAE.metrics = measure_everything(SSAE, train_loader, device, return_sqr=True)
ASAE.metrics = measure_everything(ASAE, train_loader, device, return_sqr=True)
print(f"Energy per concept: {ASAE.metrics.E[torch.argsort(ASAE.metrics.E, descending=True)[:10]]}")
print(f"Frequency per concept: {ASAE.metrics.f[torch.argsort(ASAE.metrics.E, descending=True)[:10]]}")
print(f"Modality score per concept: {ASAE.metrics.mu[torch.argsort(ASAE.metrics.E, descending=True)[:10]]}")

In [ ]:
print(f"SSAE metrics: {SSAE.metrics}")
print(f"ASAE metrics: {ASAE.metrics}")

In [ ]:
SSAE_bridge = {
    "bridge_sigma" : measure_bridge_sigma(SSAE, train_loader, SSAE.metrics.E, SSAE.metrics.E_Img, SSAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False).cpu().detach(),
    # "bridge_sigma_null_C" : measure_bridge_sigma(SSAE, train_loader, SSAE.metrics.E, SSAE.metrics.E_Img, SSAE.metrics.E_Txt, device, center=False, normalize=False, null_C=True, null_D=False).cpu().detach(),
    # "bridge_sigma_null_D" : measure_bridge_sigma(SSAE, train_loader, SSAE.metrics.E, SSAE.metrics.E_Img, SSAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=True).cpu().detach(),
    # "bridge_sigma_null" : measure_bridge_sigma(SSAE, train_loader, SSAE.metrics.E, SSAE.metrics.E_Img, SSAE.metrics.E_Txt, device, center=False, normalize=False, null_C=True, null_D=True).cpu().detach(),
}
# _prune_bridge(SSAE_bridge["bridge_sigma"], SSAE_bridge["bridge_sigma_null"])
# _prune_bridge(SSAE_bridge["bridge_sigma_null_C"], SSAE_bridge["bridge_sigma_null"])
# _prune_bridge(SSAE_bridge["bridge_sigma_null_D"], SSAE_bridge["bridge_sigma_null"])
ASAE_bridge = {
    "bridge_sigma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False).cpu().detach(),
    # "bridge_sigma_null_C" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=True, null_D=False).cpu().detach(),
    # "bridge_sigma_null_D" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=True).cpu().detach(),
    # "bridge_sigma_null" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=True, null_D=True).cpu().detach(),
}
# _prune_bridge(ASAE_bridge["bridge_sigma"], ASAE_bridge["bridge_sigma_null"])
# _prune_bridge(ASAE_bridge["bridge_sigma_null_C"], ASAE_bridge["bridge_sigma_null"])
# _prune_bridge(ASAE_bridge["bridge_sigma_null_D"], ASAE_bridge["bridge_sigma_null"])

In [ ]:
SSAE_bridge_gamma = {
    "bridge_gamma" : measure_bridge_sigma(SSAE, train_loader, SSAE.metrics.E, SSAE.metrics.E_Img, SSAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False, weight_type="OT").cpu().detach(),
    # "bridge_gamma_null_C" : measure_bridge_sigma(SSAE, train_loader, SSAE.metrics.E, SSAE.metrics.E_Img, SSAE.metrics.E_Txt, device, center=False, normalize=False, null_C=True, null_D=False, weight_type="OT").cpu().detach(),
    # "bridge_gamma_null_D" : measure_bridge_sigma(SSAE, train_loader, SSAE.metrics.E, SSAE.metrics.E_Img, SSAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=True, weight_type="OT").cpu().detach(),
    # "bridge_gamma_null" : measure_bridge_sigma(SSAE, train_loader, SSAE.metrics.E, SSAE.metrics.E_Img, SSAE.metrics.E_Txt, device, center=False, normalize=False, null_C=True, null_D=True, weight_type="OT").cpu().detach(),
}
# _prune_bridge(SSAE_bridge_gamma["bridge_gamma"], SSAE_bridge_gamma["bridge_gamma_null"])
# _prune_bridge(SSAE_bridge_gamma["bridge_gamma_null_C"], SSAE_bridge_gamma["bridge_gamma_null"])
# _prune_bridge(SSAE_bridge_gamma["bridge_gamma_null_D"], SSAE_bridge_gamma["bridge_gamma_null"])
ASAE_bridge_gamma = {
    "bridge_gamma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False, weight_type="OT").cpu().detach(),
    # "bridge_gamma_null_C" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=True, null_D=False, weight_type="OT").cpu().detach(),
    # "bridge_gamma_null_D" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=True, weight_type="OT").cpu().detach(),
    # "bridge_gamma_null" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=True, null_D=True, weight_type="OT").cpu().detach(),
}
# _prune_bridge(ASAE_bridge_gamma["bridge_gamma"], ASAE_bridge_gamma["bridge_gamma_null"])
# _prune_bridge(ASAE_bridge_gamma["bridge_gamma_null_C"], ASAE_bridge_gamma["bridge_gamma_null"])
# _prune_bridge(ASAE_bridge_gamma["bridge_gamma_null_D"], ASAE_bridge_gamma["bridge_gamma_null"])

# fuse SSAE bridge and SSAE bridge_gamma
for key in SSAE_bridge_gamma:
    SSAE_bridge[key] = SSAE_bridge_gamma[key]
for key in ASAE_bridge_gamma:
    ASAE_bridge[key] = ASAE_bridge_gamma[key]

torch.save(SSAE_bridge, f"./checkpoints/{SSAE_name.replace('.pt', '_bridge.pt')}")
torch.save(ASAE_bridge, f"./checkpoints/{ASAE_name.replace('.pt', '_bridge.pt')}")


In [ ]:
get_bridge_norms(SSAE_bridge["bridge_sigma"], SSAE.metrics.mu, SSAE.metrics.E, SSAE_name, SSAE)
get_bridge_norms(ASAE_bridge["bridge_sigma"], ASAE.metrics.mu, ASAE.metrics.E, ASAE_name, ASAE)

In [ ]:
print(f"SSAE metrics: {SSAE.metrics}")
print(f"ASAE metrics: {ASAE.metrics}")

In [ ]:
plot_energy_per_concept(SSAE.metrics.E, SSAE.metrics.f, SSAE_name)
plot_energy_per_concept(ASAE.metrics.E, ASAE.metrics.f, ASAE_name)
plot_energy_cdf(SSAE.metrics.E, SSAE_name)
plot_energy_cdf(ASAE.metrics.E, ASAE_name)

In [ ]:
plot_E_vs_E_B(SSAE.metrics.E, SSAE.metrics.E_B, SSAE_name)
plot_E_vs_E_B(ASAE.metrics.E, ASAE.metrics.E_B, ASAE_name)

## 4. General Metrics

#### TODO : store them all in a json file.
Do not append the "beta..." at the end of the file name, but add it as a key in the json file. Also include other metrics that are not for one specific dict, e.g. transport between SSAE and ASAE, modality gap measures, etc.

#### TODO : average all these metrics across many runs of SAE training for better statistics.

In [ ]:
# Get metrics :

def get_sparse_reconstruction_metrics(sae, train_loader):
    l2, r2 = metrics.l2_r2(sae, train_loader)
    print(f"l2: {l2}, r2: {r2}")
    l0, l1 = metrics.l0_l1(sae, train_loader)
    print(f"l0: {l0}, l1: {l1}")
    dead = metrics.dead_features(sae, train_loader)
    print(f"Dead features ratio: {dead}")
    print(f"Dead features %: {dead * 100}")
    sae.metrics._add_more({
        "l2": l2,
        "r2": r2,
        "l0": l0,
        "l1": l1,
        "dead": dead,
    }, save=True)

def get_C_metrics(sae, train_loader, top_k):
    # TODO : add mu_fidelity
    C_insertion, curve = metrics.C_insertion(sae, train_loader, L0=top_k, return_curve=True, metric="l2")
    print(C_insertion)
    my_own_C_insertion = 0
    for i in range(1, len(curve)):
        my_own_C_insertion += curve[i]
    my_own_C_insertion /= len(curve)
    print(my_own_C_insertion)
    # plt.plot(curve[:])
    # plt.title("C_insertion")
    # plt.xlabel("Number of concepts")
    # plt.ylabel("C_insertion")
    # plt.show()
    C_deletion, deletion_curve = metrics.C_deletion(sae, train_loader, L0=top_k, return_curve=True, metric="l2")
    print(C_deletion)
    my_own_C_deletion = 0
    for i in range(1, len(deletion_curve)):
        my_own_C_deletion += deletion_curve[i]
    my_own_C_deletion /= len(deletion_curve)
    print(my_own_C_deletion)
    # plt.plot(deletion_curve[:])
    # plt.title("C_deletion")
    # plt.xlabel("Number of concepts")
    # plt.ylabel("C_deletion")
    # plt.show()
    sae.metrics._add_more({
        "C_insertion": C_insertion,
        "C_deletion": C_deletion,
    }, save=True)

def emd_alignment():
    # OT between the two dictionaries
    D0 = SSAE.dictionary._fused_dictionary
    D05 = ASAE.dictionary._fused_dictionary
    
    a = SSAE.metrics.E.cpu().numpy() / SSAE.metrics.E.sum().cpu().numpy()
    b = ASAE.metrics.E.cpu().numpy() / ASAE.metrics.E.sum().cpu().numpy()

    c = metrics.Wasserstein(D0, D05, a, b, metric='cosim')
    c_null = metrics.Wasserstein(D0, D05, a, b[np.random.permutation(len(b))], metric='cosim')
    print(c)
    print(c_null)
    SSAE.metrics._add_more({"Transport cost to Aligned SAE": c, "Transport cost null": c_null}, save=True)
    ASAE.metrics._add_more({"Transport cost to Standard SAE": c, "Transport cost null": c_null}, save=True)

def get_stability_beta(train_loader, beta):
    # OT between similarly trained dictionaries
    T = 10
    wasss = []
    wasserstein_nulls = []
    for i in range(T):
        if UseTOPK:
            top_k_individual = top_k * 2
            sae1 = overcomplete.sae.BatchTopKSAE(d_model, nb_concepts=nb_concepts, top_k=top_k_individual*batch_size, device=device)
            sae2 = overcomplete.sae.BatchTopKSAE(d_model, nb_concepts=nb_concepts, top_k=top_k_individual*batch_size, device=device)
            criterion_1 = lambda *args, **kwargs: overcomplete.sae.losses.top_k_auxiliary_loss(*args, **kwargs, penalty=0.1)
        else:
            sae1 = overcomplete.sae.MpSAE(d_model, nb_concepts=nb_concepts, k=top_k, device=device)
            sae2 = overcomplete.sae.MpSAE(d_model, nb_concepts=nb_concepts, k=top_k, device=device)
            criterion_1 = lambda *args, **kwargs: overcomplete.sae.losses.mse_l1(*args, **kwargs, penalty=0.0)
                
        criterion_2 = lambda *args, **kwargs: losses.alignment_penalty(*args, **kwargs, penalty=beta, alignment_metric='cosim')
        criterion = lambda *args, **kwargs: criterion_1(*args, **kwargs) + criterion_2(*args, **kwargs)

        sae1.to(device)
        sae2.to(device)
        sae1.train()
        sae2.train()
        optimizer1 = torch.optim.Adam(sae1.parameters(), lr=5e-4)
        optimizer2 = torch.optim.Adam(sae2.parameters(), lr=5e-4)

        steps_per_epoch = len(train_loader)
        scheduler1 = torch.optim.lr_scheduler.OneCycleLR(
            optimizer1, max_lr=lr, total_steps=epochs * steps_per_epoch,
        )
        scheduler2 = torch.optim.lr_scheduler.OneCycleLR(
            optimizer2, max_lr=lr, total_steps=epochs * steps_per_epoch,
        )
        logs1 = train.train_multimodal_sae(
            sae1, train_loader, criterion, optimizer1, scheduler=scheduler1, nb_epochs=epochs, device=device,
            monitoring=1, verbose=False,
            checkpoint_path=None,
        )
        logs2 = train.train_multimodal_sae(
            sae2, train_loader, criterion, optimizer2, scheduler=scheduler2, nb_epochs=epochs, device=device,
            monitoring=1, verbose=False,
            checkpoint_path=None,
        )
        metrics1 = measure_everything(sae1, train_loader, device, return_sqr=True)
        metrics2 = measure_everything(sae2, train_loader, device, return_sqr=True)
        E1 = metrics1.E
        E2 = metrics2.E
        a = E1.cpu().numpy() / np.sum(E1.cpu().numpy())
        b = E2.cpu().numpy() / np.sum(E2.cpu().numpy())

        D1 = sae1.dictionary._fused_dictionary
        D2 = sae2.dictionary._fused_dictionary

        wasserstein = metrics.Wasserstein(D1, D2, a, b, metric="cosim")
        wasserstein_null = metrics.Wasserstein(D1, D2, a, b[np.random.permutation(len(b))], metric="cosim")
        print(f"Wasserstein distance: {wasserstein}")
        print(f"Wasserstein distance null: {wasserstein_null}")
        wasss.append(wasserstein)
        wasserstein_nulls.append(wasserstein_null)
    wasss = np.array(wasss)
    wasserstein_nulls = np.array(wasserstein_nulls)
    return {
        "stability (mean, std)": (wasss.mean(), wasss.std()),
        "stability null (mean, std)": (wasserstein_nulls.mean(), wasserstein_nulls.std()),
    }

def get_D_structure_metrics(sae):
    D = sae.dictionary._fused_dictionary
    w = sae.metrics.E.unsqueeze(1)
    
    sae.metrics._add_more({
        "stable_rank": metrics.stable_rank(D),
        "stable_rank_weighted": metrics.stable_rank(D, w=w),
        "effective_rank": metrics.effective_rank(D),
        "effective_rank_weighted": metrics.effective_rank(D, w=w),
        "coherence": metrics.coherence(D),
    }, save=True)

def get_Z_structure_metrics(sae, train_loader):
    D = sae.dictionary._fused_dictionary
    sae.metrics._add_more({
        "connectivity": metrics.connectivity(sae, data_loader=train_loader),
        "negative_interference": metrics.negative_interference(sae, data_loader=train_loader, D=D),
    }, save=True)


In [ ]:
get_sparse_reconstruction_metrics(SSAE, train_loader)
get_sparse_reconstruction_metrics(ASAE, train_loader)
get_C_metrics(SSAE, train_loader)
get_C_metrics(ASAE, train_loader)

emd_alignment()
# SSAE.metrics._add_more(get_stability_beta(train_loader, beta=0e-4), save=True)
# ASAE.metrics._add_more(get_stability_beta(train_loader, beta=5e-4), save=True)

get_D_structure_metrics(SSAE)
get_D_structure_metrics(ASAE)

get_Z_structure_metrics(SSAE, train_loader)
get_Z_structure_metrics(ASAE, train_loader)

In [ ]:
# # Global C-insertion :

# # compute the R^2 score for images, text, and both images and text.
# # Do it by keeping only the topk most activated concepts.

# @torch.no_grad()
# def compute_r2_score(sae, energy, data_loader, device, idxs):
#     sae.eval()

#     sae_dict = {}
#     img_score = {}
#     txt_score = {}
#     total_score = {}

#     for idx in idxs:
#         sae_truncated = copy.deepcopy(sae)
#         D = torch.zeros_like(sae.dictionary._fused_dictionary)
#         D[idx] = sae.dictionary._fused_dictionary[idx]
#         sae_truncated.dictionary._fused_dictionary = D
#         sae_dict[idx] = sae_truncated

#         img_score[idx] = 0
#         txt_score[idx] = 0
#         total_score[idx] = 0

#     n_images = 0
#     n_texts = 0
#     n_inputs = 0

#     for image_features, text_features in tqdm(data_loader):
#         n_images += 1 # image_features.shape[0]
#         n_texts += 1 # text_features.shape[0]

#         image_features = image_features.to(device)
#         text_features = text_features.to(device)

#         for idx in idxs:
#             # Forward pass through the model
#             sae_truncated = sae_dict[idx]
#             _, _, x_hat_image_truncated = sae_truncated(image_features)
#             _, _, x_hat_text_truncated = sae_truncated(text_features)
            
#             # Compute the R^2 score for images and text
#             img_score[idx] += r2_score(image_features, x_hat_image_truncated).item()
#             txt_score[idx] += r2_score(text_features, x_hat_text_truncated).item()

#     n_inputs = n_images + n_texts
#     for idx in idxs:
#         total_score[idx] = (img_score[idx] + txt_score[idx]) / (n_inputs)
#         img_score[idx] /= n_images
#         txt_score[idx] /= n_texts

#     return img_score, txt_score, total_score

In [ ]:
# Global C-insertion :

# compute the R^2 score for images, text, and both images and text.
# Do it by keeping only the topk most activated concepts.

@torch.no_grad()
def compute_r2_score(sae, data_loader, device, idxs):
    sae.eval()

    img_score = {idx: 0 for idx in idxs}
    txt_score = {idx: 0 for idx in idxs}
    total_score = {idx: 0 for idx in idxs}

    n_images = 0
    n_texts = 0

    for image_features, text_features in tqdm(data_loader):
        n_images += 1 # image_features.shape[0]
        n_texts += 1 # text_features.shape[0]

        image_features = image_features.to(device)
        text_features = text_features.to(device)

        _, image_codes = sae.encode(image_features)
        _, text_codes = sae.encode(text_features)

        for idx in idxs:
            truncated_image_codes = image_codes.clone()
            truncated_image_codes[:, ~idx] = 0
            x_hat_image_truncated = sae.decode(truncated_image_codes)

            truncated_text_codes = text_codes.clone()
            truncated_text_codes[:, ~idx] = 0
            x_hat_text_truncated = sae.decode(truncated_text_codes)
            
            # Compute the R^2 score for images and text
            img_score[idx] += r2_score(image_features, x_hat_image_truncated).item()
            txt_score[idx] += r2_score(text_features, x_hat_text_truncated).item()

    n_inputs = n_images + n_texts
    for idx in idxs:
        total_score[idx] = (img_score[idx] + txt_score[idx]) / (n_inputs)
        img_score[idx] /= n_images
        txt_score[idx] /= n_texts

    return img_score, txt_score, total_score

def plot_r2_vs_energy(sae, sae_name, E):
    # This is the global version of C-insertion.
    idxs = []

    energy_perm = torch.argsort(E, descending=True)
    energy_perm_inv = torch.argsort(energy_perm)

    thresholds = torch.linspace(E.min() + 1e-10, 1, steps=100)
    # thresholds = torch.logspace(-6, 0, steps=60)

    # plot energy vs index, cumulative sum vs index :
    sorted_energy = E[energy_perm]
    sorted_energy = sorted_energy / sorted_energy.sum()
    cumsum_energy = 1 - sorted_energy.cumsum(dim=0)

    # find thresholds index in cumsum_energy
    thresholds_idx = []
    for threshold in thresholds:
        idx = (cumsum_energy < threshold).nonzero(as_tuple=True)[0][0]
        thresholds_idx.append(idx.item())
    thresholds_idx = torch.tensor(thresholds_idx)

    img_score, txt_score, total_score = compute_r2_score(sae, train_loader, device, idxs)
    id_line = torch.linspace(0, 1, steps=100)
    plt.figure(figsize=(10, 6))
    plt.plot(id_line.cpu().numpy(), id_line.cpu().numpy(), '--', color='black', label='y=x')
    plt.plot(1 - thresholds.cpu().numpy(), torch.tensor([r2 for r2 in total_score.values()]).cpu().numpy(), label='R^2 Score', color='blue')
    plt.xlabel('Energy mass')
    plt.ylabel('R^2 Score')
    plt.title('R^2 Score vs Energy mass (E) - most energetic concepts containing E of energy')
    plt.legend()
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/energy/r2_score.png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/energy/r2_score.pdf")
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(torch.arange(E.shape[0]) + 1, 1-cumsum_energy.cpu().numpy(), label='Cumulative Energy', color='blue')
    plt.plot(torch.tensor(thresholds_idx) + 1, torch.tensor([r2 for r2 in total_score.values()]).cpu().numpy(), label='R^2 Score', color='red')
    plt.xlabel('index')
    plt.ylabel('Mass & R^2 Score')
    plt.title('R^2 & Energy mass contained in most energetic concepts')
    plt.legend()
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/energy/r2_energy_vs_index.png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/energy/r2_energy_vs_index.pdf")
    plt.show()


In [ ]:
# plot_r2_vs_energy(SSAE, SSAE_name, SSAE.metrics.E)
# plot_r2_vs_energy(ASAE, ASAE_name, ASAE.metrics.E)

## 5. Modality specific Metrics

#### 5.1. Modality Scores

In [ ]:
plot_modality_energy(SSAE, SSAE_name)
# plot_modality_energy(SSAE, SSAE_name, mu_B=True)

In [ ]:
plot_modality_energy(ASAE, ASAE_name)
# plot_modality_energy(ASAE, ASAE_name, mu_B=True)

In [ ]:
mu_vs_mu_B(SSAE, SSAE_name)
mu_vs_mu_B(ASAE, ASAE_name)

In [ ]:
# Modality vs R^2
# compute the R^2 score for images, text, and both images and text.
# Do it by keeping only the bimodal concepts ((modality-0.5).abs() < 0.5 - eps)

def R2_vs_modality_score(sae, sae_name, E, mu):
    # C-insertion like
    idxs = []
    epss = torch.linspace(0, 0.5, 20)

    for i, eps in enumerate(epss):
        bimodal_mask = (mu - 0.5).abs() < eps
        bimodal_mask = bimodal_mask.cpu()
        idxs.append(bimodal_mask)

    img_score, txt_score, total_score = compute_r2_score(sae, train_loader, device, idxs)
    # print(f"Image R^2 score: {img_score}")
    # print(f"Text R^2 score: {txt_score}")
    # print(f"R^2 score: {total_score}")

    # compute AUC or that :
    MAUC_insertion = 0
    for i in range(len(epss)):
        MAUC_insertion += (total_score[idxs[i]])
    MAUC_insertion /= len(epss)
    print(f"mu_insertion: {MAUC_insertion}")
    sae.metrics._add_more({
        "mu_insertion": MAUC_insertion,
    }, save=True)

    # plot r2 w.r.t. epsilon
    id_line = torch.linspace(0, 1, steps=100)
    plt.figure(figsize=(10, 6))
    # plt.plot(id_line.cpu().numpy(), id_line.cpu().numpy(), '--', color='black', label='y=x')
    plt.plot(epss.cpu().numpy(), torch.tensor([r2 for r2 in total_score.values()]).cpu().numpy(), label='R^2 Score', color='blue')
    plt.xlabel('Epsilon')
    plt.ylabel('R^2 Score')
    plt.title('R^2 Score vs Bimodal Threshold')
    plt.legend()
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/modality/r2_modality_insertion.png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/modality/r2_modality_insertion.pdf")
    plt.show()

    # C-deletion like
    idxs = []
    epss = torch.linspace(0, 0.5, 20)

    for i, eps in enumerate(epss):
        bimodal_mask = (mu - 0.5).abs() >= eps
        bimodal_mask = bimodal_mask.cpu()
        idxs.append(bimodal_mask)

    img_score, txt_score, total_score = compute_r2_score(sae, train_loader, device, idxs)
    # print(f"Image R^2 score: {img_score}")
    # print(f"Text R^2 score: {txt_score}")
    # print(f"R^2 score: {total_score}")

    # compute AUC or that :
    MAUC_deletion = 0
    for i in range(len(epss)):
        MAUC_deletion += (total_score[idxs[i]])
    MAUC_deletion /= len(epss)
    print(f"mu_deletion: {MAUC_deletion}")
    sae.metrics._add_more({
        "mu_deletion": MAUC_deletion,
    }, save=True)

    # plot r2 w.r.t. epsilon
    id_line = torch.linspace(0, 1, steps=100)
    plt.figure(figsize=(10, 6))
    # plt.plot(id_line.cpu().numpy(), id_line.cpu().numpy(), '--', color='black', label='y=x')
    plt.plot(epss.cpu().numpy(), torch.tensor([r2 for r2 in total_score.values()]).cpu().numpy(), label='R^2 Score', color='blue')
    plt.xlabel('Epsilon')
    plt.ylabel('R^2 Score')
    plt.title('R^2 Score vs Bimodal Threshold')
    plt.legend()
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/modality/r2_modality_deletion.png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/modality/r2_modality_deletion.pdf")
    plt.show()

In [ ]:
# R2_vs_modality_score(SSAE, SSAE_name, SSAE.metrics.E, SSAE.metrics.mu)
# R2_vs_modality_score(ASAE, ASAE_name, ASAE.metrics.E, ASAE.metrics.mu)

#### 5.2. Bridge

In [ ]:
plot_bridges(SSAE_bridge, SSAE, SSAE_name, SSAE.metrics.mu)
plot_bridges(ASAE_bridge, ASAE, ASAE_name, ASAE.metrics.mu)

In [ ]:
get_rho(SSAE, SSAE.metrics.bridge_mass_vs_eps_gamma, normalize=True)
get_rho(ASAE, ASAE.metrics.bridge_mass_vs_eps_gamma, normalize=True)

get_rho(SSAE, SSAE.metrics.bridge_mass_vs_eps_sigma, normalize=False)
get_rho(ASAE, ASAE.metrics.bridge_mass_vs_eps_sigma, normalize=False)

#### Null models
- Randomly shuffle C the correlation matrix
    - during computation, shuffle z_image and z_text
    - Are the observed co-activation patterns stronger than chance, given the same geometry?
- Randomly shuffle D the features
    - v1 : sample D' uniformly on the unit sphere - completely random and isotropic, losing all information about the geometry
    - v2 : permute D - preserves the geometry but loses semantic information since not aligned with C anymore.
    - Are observed geometric alignments meaningful, given activation behavior?
- Randomly shuffle the features and the correlation matrix
    - The complete absence of any structure — the fully unstructured baseline.

## 6. Geometric Analysis

#### 6.1. Cosim Histograms & Data Projections

In [ ]:
@torch.no_grad()
def plot_cosim_histogram(I, T, D, sae_name, subtitle=""):
    # Baseline (random isotropic Gaussian)
    Rd = torch.randn_like(D)
    Rd /= Rd.norm(dim=-1, keepdim=True)
    J = Rd[torch.randperm(Rd.shape[0], device='cpu')]
    Rd_baseline = torch.cosine_similarity(Rd, J, dim=1).cpu().numpy()

    # means
    mu_I = I.mean(dim=0)
    mu_T = T.mean(dim=0)
    mu_D = D.mean(dim=0)

    def _aux_cosim(A, B, mu_A=0, mu_B=0):
        A_rd = A[torch.randperm(A.shape[0], device='cpu')]
        B_rd = B[torch.randperm(B.shape[0], device='cpu')]
        AA = torch.cosine_similarity(A - mu_A, A_rd - mu_A, dim=1).cpu().numpy()
        BB = torch.cosine_similarity(B - mu_B, B_rd - mu_B, dim=1).cpu().numpy()
        AB_aligned = torch.cosine_similarity(A - mu_A, B - mu_B, dim=1).cpu().numpy()
        AB_rd = torch.cosine_similarity(A - mu_A, B_rd - mu_B, dim=1).cpu().numpy()
        return AA, BB, AB_aligned, AB_rd

    II, TT, IT_aligned, IT_rd = _aux_cosim(I, T)
    II_D, TT_D, IT_aligned_D, IT_rd_D = _aux_cosim(I, T, mu_A=mu_D, mu_B=mu_D)
    II_m, TT_m, IT_aligned_m, IT_rd_m = _aux_cosim(I, T, mu_A=mu_I, mu_B=mu_T)

    def plot_histograms(data_dict, title):
        plt.figure(figsize=(10, 5))
        all_data = np.concatenate(list(data_dict.values()))
        bins = np.linspace(all_data.min(), all_data.max(), 1000)  # 50 bins
        for label, data in data_dict.items():
            plt.hist(data, bins=bins, alpha=0.6, label=label, density=True)
        # plt.axvline(threshold, color='red', linestyle='--', linewidth=1, label='Threshold')
        plt.title(title)
        plt.xlabel('Cosine Similarity')
        plt.ylabel('Density')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        os.makedirs(f"./figures/{sae_name.replace('.pt', '')}/geometry", exist_ok=True)
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/{title.replace(' ', '_').lower()}_{subtitle.replace(' ', '_').lower()}.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/{title.replace(' ', '_').lower()}_{subtitle.replace(' ', '_').lower()}.pdf")
        plt.show()

    plot_histograms({
        "Random (N(0,I))": Rd_baseline,
        "I-I": II,
        "I-T": IT_rd,
        "T-T": TT,
        "I-T (aligned)": IT_aligned,
    }, "Uncentered Cosine Similarity Distributions")

    plot_histograms({
        "Random (N(0,I))": Rd_baseline,
        "I-I": II_D,
        "I-T": IT_rd_D,
        "T-T": TT_D,
        "I-T (aligned)": IT_aligned_D,
    }, "Centered Cosine Similarity Distributions")

    plot_histograms({
        "Random (N(0,I))": Rd_baseline,
        "I-I": II_m,
        "I-T": IT_rd_m,
        "T-T": TT_m,
        "I-T (aligned)": IT_aligned_m,
    }, "Modality-Centered Cosine Similarity Distributions")

    # Label ZOULOU_69 : threshold for
    # COCO : 0.238
    # CC3M : /
    # LAION 400M : None - already thresholded at 0.3. 


In [ ]:
I = train_loader.dataset.tensors[0]
T = train_loader.dataset.tensors[1]
D_ict = torch.cat([I, T], dim=0)
I_mu = I.mean(dim=0)
T_mu = T.mean(dim=0)
D_mu = D_ict.mean(dim=0)
# plot_cosim_histogram(I, T, D_ict, SSAE_name)
# plot_cosim_histogram(I, T, D_ict, ASAE_name)

In [ ]:
def plot_pca(D, d1, d2, I_mu, T_mu, D_mu, title, save_title, sae_name):
    modality_color = {
        "bimodal": np.array([91/255, 40/255, 68/255]),
        "image": np.array([128/255, 230/255, 255/255]),
        "text": np.array([255/255, 205/255, 102/255]),
    }
    def make_alpha_cmap(base_color, name):
        r, g, b = base_color
        return LinearSegmentedColormap.from_list(
            name,
            [(r, g, b, 0.0), (r, g, b, 1.0)]
        )

    I_cmap = make_alpha_cmap(modality_color["image"], "I")
    T_cmap = make_alpha_cmap(modality_color["text"], "T")
    # Plot the projections, colored by modality (D[:N//2] = I, D[N//2:] = T)
    plt.figure(figsize=(10, 7))
    plt.hexbin(D[:D.shape[0]//2].cpu().numpy() @ d1.reshape(-1, 1), D[:D.shape[0]//2].cpu().numpy() @ d2.reshape(-1, 1), gridsize=50, cmap=I_cmap, bins='log')
    plt.colorbar(label='Image Counts')
    plt.hexbin(D[D.shape[0]//2:].cpu().numpy() @ d1.reshape(-1, 1), D[D.shape[0]//2:].cpu().numpy() @ d2.reshape(-1, 1), gridsize=50, cmap=T_cmap, bins='log')
    plt.colorbar(label='Text Counts')
    plt.axhline(0, color='k', linestyle='--', linewidth=1)
    plt.axvline(0, color='k', linestyle='--', linewidth=1)

    # Plot all the means of the modalities
    plt.scatter(D_mu.cpu().numpy() @ d1.reshape(-1, 1), D_mu.cpu().numpy() @ d2.reshape(-1, 1), color='k', marker='x', s=100, label='mu_D, mu_I, mu_T')
    plt.scatter(I_mu.cpu().numpy() @ d1.reshape(-1, 1), I_mu.cpu().numpy() @ d2.reshape(-1, 1), color='k', marker='x', s=100)
    plt.scatter(T_mu.cpu().numpy() @ d1.reshape(-1, 1), T_mu.cpu().numpy() @ d2.reshape(-1, 1), color='k', marker='x', s=100)

    plt.xlabel('d1')
    plt.ylabel('d2')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    os.makedirs(f"./figures/{sae_name.replace('.pt', '')}/geometry", exist_ok=True)
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/{save_title}.png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/{save_title}.pdf")
    plt.show()

def fit_and_plot_pca(I, T, D, sae_name, save_title='data_pca_pc1_pc2'):
    # Fit PCA on D
    D_np = D.cpu().numpy()
    pca = PCA(n_components=2)
    pca.fit(D_np)
    mean = pca.mean_
    d1 = pca.components_[0]
    d2 = pca.components_[1]
    D_pca = pca.transform(D_np)

    I_mu = I.mean(dim=0)
    T_mu = T.mean(dim=0)
    D_mu = D.mean(dim=0)

    plot_pca(D-mean, d1, d2, I_mu-mean, T_mu-mean, D_mu-mean, title='Projection of D on d1 = PC1 and d2 = PC2', save_title=save_title, sae_name=sae_name)


In [ ]:
# Plot "PCA"s of D, but not on the top 2 PCs, instead on (1) D_mu, (2) I_mu - T_mu

d1 = D_mu.cpu().numpy()
d1 = d1 / np.linalg.norm(d1)
d2 = I_mu.cpu().numpy() - T_mu.cpu().numpy()
d2 = d2 / np.linalg.norm(d2)

plot_pca(D_ict, d1, d2, I_mu, T_mu, D_mu, title='Projection of D on d1 = D_mu and d2 = I_mu - T_mu', save_title='data_pca_d1_d2', sae_name=SSAE_name)
plot_pca(D_ict, d1, d2, I_mu, T_mu, D_mu, title='Projection of D on d1 = D_mu and d2 = I_mu - T_mu', save_title='data_pca_d1_d2', sae_name=ASAE_name)

fit_and_plot_pca(I, T, D_ict, SSAE_name)
fit_and_plot_pca(I, T, D_ict, ASAE_name)


In [ ]:
# remove unimodal concepts, then plot PCA similar to above : D_bimodal, I_bimodal, T_bimodal

@torch.no_grad()
def get_bimodal_data(sae, train_loader, modality_score, device, complement=False, use_E_B=False, eps=0.05, tau=0.5):
    I_bimodal = torch.zeros_like(train_loader.dataset.tensors[0])
    T_bimodal = torch.zeros_like(train_loader.dataset.tensors[1])

    with torch.no_grad():
        sae.eval()

        n_images = 0
        n_texts = 0
        
        if not use_E_B:
            mask = (modality_score >= eps) & (modality_score <= 1 - eps)
        else:
            E_B = sae.metrics.E_B
            E = sae.metrics.E
            ratio = E_B / (E + 1e-8)
            mask = (ratio >= tau)
        if complement:
            mask = ~mask
        idx = mask.cpu()

        for image_features, text_features in tqdm(train_loader):
            n_i = image_features.shape[0]
            n_t = text_features.shape[0]

            image_features = image_features.to(device)
            _, image_codes = sae.encode(image_features)
            truncated_image_codes = image_codes.clone()
            truncated_image_codes[:, ~idx] = 0
            x_hat_image_truncated = sae.decode(truncated_image_codes)

            I_bimodal[n_images:n_images+n_i] = x_hat_image_truncated
            
            if text_features.dtype == image_features.dtype:
                text_features = text_features.to(device)
                _, text_codes = sae.encode(text_features)
                truncated_text_codes = text_codes.clone()
                truncated_text_codes[:, ~idx] = 0
                x_hat_text_truncated = sae.decode(truncated_text_codes)
                
                T_bimodal[n_images:n_images+n_i] = x_hat_text_truncated

            n_images += n_i
            n_texts += n_t

        n_inputs = n_images + n_texts
    if text_features.dtype == image_features.dtype:
        D_bimodal = torch.cat([I_bimodal, T_bimodal], dim=0)
    else:
        D_bimodal = None
    return I_bimodal, T_bimodal, D_bimodal

In [ ]:
I_bi_S, T_bi_S, B_bi_S = get_bimodal_data(SSAE, train_loader, SSAE.metrics.mu, device)
# plot_cosim_histogram(I_bi_S, T_bi_S, B_bi_S, SSAE_name, subtitle="Unimodal concepts removed")
I_bi_A, T_bi_A, B_bi_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device)
# plot_cosim_histogram(I_bi_A, T_bi_A, B_bi_A, ASAE_name, subtitle="Unimodal concepts removed")

In [ ]:
fit_and_plot_pca(I_bi_S, T_bi_S, B_bi_S, SSAE_name, save_title='data_proj_pca_pc1_pc2')
fit_and_plot_pca(I_bi_A, T_bi_A, B_bi_A, ASAE_name, save_title='data_proj_pca_pc1_pc2')

#### 6.2. Dictionary Orthogonality - $E = D^T$

In [ ]:
# # Cosim between E^T_i and D_i (weighted and unweighted by energy)

# @torch.no_grad()
# def compute_cosim(E, D, energy=None):
#     dot_product = torch.einsum('ij,ji->i', E, D)  # Shape: (nb_concepts, d_model) * (d_model, nb_concepts) -> (nb_concepts,)
#     norm_E = torch.norm(E, p=2, dim=1)  # Shape: (nb_concepts,)
#     norm_D = torch.norm(D, p=2, dim=0)  # Shape: (nb_concepts,)
#     cosim = dot_product / (norm_E * norm_D + 1e-6)  # Shape: (nb_concepts,)

#     # print mean and std of dot product and cosim :
#     dot_mean = dot_product.mean()
#     dot_std = dot_product.std()
#     print(f"Unweighted dot product mean: {dot_product.mean()}, std: {dot_product.std()}")
#     print(f"Unweighted cosim mean: {cosim.mean()}, std: {cosim.std()}")
#     weighted_dot_product = dot_product * energy
#     weighted_dot_product_mean = weighted_dot_product.sum() / energy.sum()
#     weighted_dot_product_std = torch.sqrt(((dot_product - weighted_dot_product_mean)**2 * energy).sum() / energy.sum())
#     print(f"\n\nWeighted dot product mean: {weighted_dot_product_mean}, {weighted_dot_product_std}")
#     weighted_cosim = cosim * energy
#     weighted_cosim_mean = weighted_cosim.sum() / energy.sum()
#     weighted_cosim_std = torch.sqrt(((cosim - weighted_cosim_mean)**2 * energy).sum() / energy.sum())
#     print(f"Weighted cosim mean: {weighted_cosim_mean}, {weighted_cosim_std}")

#     # plot histograms of dot product and cosim
#     plt.figure(figsize=(10, 6))
#     plt.hist(dot_product.cpu().numpy(), bins=100, range=(-1, 1), color='blue', alpha=0.7, label='Dot Product')
#     plt.title("Distribution of Dot Product")
#     plt.xlabel("Dot Product")
#     plt.ylabel("count")
#     plt.yscale("log")
#     plt.grid(axis='y', alpha=0.75)
#     plt.legend()
#     # Add mean and std information to the plot
#     plt.text(0.05, 0.95, f"Mean: {dot_mean:.4f}\nStd: {dot_std:.4f}", transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))
#     plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/dot_product.png", dpi=300, bbox_inches='tight')
#     plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/dot_product.pdf")
#     plt.show()

#     # weighted dot product
#     plt.figure(figsize=(10, 6))
#     plt.hist(dot_product.cpu().numpy(), bins=100, range=(-1, 1), color='blue', alpha=0.7, label='Weighted Dot Product', weights=energy.cpu().numpy())
#     plt.title("Distribution of Weighted Dot Product")
#     plt.xlabel("Weighted Dot Product")
#     plt.ylabel("density")
#     plt.yscale("log")
#     plt.grid(axis='y', alpha=0.75)
#     plt.legend()
#     # Add mean and std information to the plot
#     plt.text(0.05, 0.95, f"Mean: {weighted_dot_product_mean:.4f}\nStd: {weighted_dot_product_std:.4f}", transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))
#     plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/dot_product_weighted.png", dpi=300, bbox_inches='tight')
#     plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/dot_product_weighted.pdf")
#     plt.show()

#     plt.figure(figsize=(10, 6))
#     plt.hist(cosim.cpu().numpy(), bins=100, range=(-1, 1), color='blue', alpha=0.7, label='Cosine Similarity')
#     plt.title("Distribution of Cosine Similarity")
#     plt.xlabel("Cosine Similarity")
#     plt.ylabel("count")
#     plt.yscale("log")
#     plt.grid(axis='y', alpha=0.75)
#     plt.legend()
#     # Add mean and std information to the plot
#     plt.text(0.05, 0.95, f"Mean: {cosim.mean():.4f}\nStd: {cosim.std():.4f}", transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))
#     plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/cosim.png", dpi=300, bbox_inches='tight')
#     plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/cosim.pdf")
#     plt.show()

#     # weighted cosim
#     plt.figure(figsize=(10, 6))
#     plt.hist(cosim.cpu().numpy(), bins=100, range=(-1, 1), color='blue', alpha=0.7, label='Weighted Cosine Similarity', weights=energy.cpu().numpy())
#     plt.title("Distribution of Weighted Cosine Similarity")
#     plt.xlabel("Weighted Cosine Similarity")
#     plt.ylabel("density")
#     plt.yscale("log")
#     plt.grid(axis='y', alpha=0.75)
#     plt.legend()
#     # Add mean and std information to the plot
#     plt.text(0.05, 0.95, f"Mean: {weighted_cosim_mean:.4f}\nStd: {weighted_cosim_std:.4f}", transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))
#     plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/cosim_weighted.png", dpi=300, bbox_inches='tight')
#     plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/cosim_weighted.pdf")
#     plt.show()

# E = sae.encoder.final_block[0].weight
# D = sae.dictionary._fused_dictionary.T
# print(E.shape, D.shape)

# compute_cosim(E, D, energy_per_concept)

#### 6.3. Dictionary Atoms Linear Separability

In [ ]:
# Visualize dictionary atoms :

def generate_visualization(sae, sae_name, modality_score, energy_per_concept, frequency_per_concept, bridge):
    # Create a UMAP of the concepts in D, colored by modality score and size by energy

    non_dead_idx = (modality_score != -1).nonzero(as_tuple=True)[0].cpu().numpy()

    D = sae.dictionary._fused_dictionary
    D_np = D.detach().cpu().numpy()[non_dead_idx]
    modality_np = modality_score.detach().cpu().numpy()[non_dead_idx]
    energy_np = energy_per_concept.detach().cpu().numpy()[non_dead_idx]
    frequency_np = frequency_per_concept.detach().cpu().numpy()[non_dead_idx]
    bridge_matrix = bridge.numpy()[non_dead_idx][:, non_dead_idx] ** 2

    non_stupid_mask = (frequency_np < 0.6)
    D_np = D_np[non_stupid_mask]
    modality_np = modality_np[non_stupid_mask]
    energy_np = energy_np[non_stupid_mask]
    bridge_matrix = bridge_matrix[non_stupid_mask][:, non_stupid_mask]

    threshold = 2e-10
    i_idx, j_idx = np.nonzero(bridge_matrix > threshold)
    line_weights = bridge_matrix[i_idx, j_idx]

    # UMAP embedding
    embedding = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine').fit_transform(D_np)

    # Line segments from i to j in the UMAP space
    lines = [ [embedding[i], embedding[j]] for i, j in zip(i_idx, j_idx) ]

    print(f"Number of lines: {len(lines)}")

    # PCA embedding
    pca = PCA(n_components=2)
    embedding_pca = pca.fit_transform(D_np)

    # Line segments from i to j in the PCA space
    lines_pca = [ [embedding_pca[i], embedding_pca[j]] for i, j in zip(i_idx, j_idx) ]

    # Make a weighted PCA :
    W = energy_np / energy_np.sum()
    mu = np.average(D_np, axis=0, weights=W)
    D_centered = D_np - mu
    C = D_centered.T @ (D_centered * W[:, None])
    eigvals, eigvecs = np.linalg.eigh(C)
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]
    embedding_weighted_pca = D_centered @ eigvecs[:, :2]

    # Line segments from i to j in the PCA space
    lines_weighted_pca = [ [embedding_weighted_pca[i], embedding_weighted_pca[j]] for i, j in zip(i_idx, j_idx) ]


    # Normalize energy for plotting size
    LOG = False
    if LOG:
        # Log scale energy normalization
        log_energy = np.log(energy_np)  # Log transform with offset to avoid log(0)
        size = log_energy - log_energy.min()
        size = 0.1 * (size) ** 2
    else:
        # Linear scale energy normalization
        size = energy_np - energy_np.min()
        size = 500 * np.sqrt((size / size[size<0.1].max()))


    # Normalize weights for line width
    # weights_norm = 10 * (line_weights - line_weights.min()) / (line_weights.max() - line_weights.min()) + 1e-2
    weights_norm = np.clip(line_weights, 0, 100)
    weights_norm = 0.05 * (np.log(weights_norm) - np.log(weights_norm.min()))


    # Plot
    plt.figure(figsize=(8, 6))
    plt.scatter(embedding[:, 0], embedding[:, 1], c=modality_np, s=size, cmap='managua')
    plt.colorbar(label='Modality Score')
    plt.title('UMAP of Dictionary Atoms')
    plt.axis('off')
    plt.tight_layout()
    xlim = plt.gca().get_xlim()
    ylim = plt.gca().get_ylim()
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/umap" + (f"_log" if LOG else "") + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/umap" + (f"_log" if LOG else "") + ".pdf")
    plt.show()

    # Plot
    blackred_cmap = LinearSegmentedColormap.from_list('blackred', [(0, 0, 0, 0.1), (1, 0, 0, 1)], N=256)
    blackred = np.zeros_like(modality_np)
    # select randomly 20 points to be red, with probability given by weights
    p = size / size.sum()
    red_indices = np.random.choice(len(modality_np), size=int(20), p=p, replace=False)
    blackred[red_indices] = 1  # Set selected indices to 1 for red color
    plt.figure(figsize=(8, 6), facecolor='#FFE5DF')
    plt.scatter(embedding[:, 0], embedding[:, 1], c=blackred, s=size, cmap=blackred_cmap, vmin=0, vmax=1)
    plt.colorbar(label='Modality Score')
    plt.title('UMAP of Dictionary Atoms')
    plt.axis('off')
    plt.tight_layout()
    xlim = plt.gca().get_xlim()
    ylim = plt.gca().get_ylim()
    # make the background of color FFC2B9 (light red)
    plt.gca().set_facecolor('#FFE5DF')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/umap_black" + (f"_log" if LOG else "") + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/umap_black" + (f"_log" if LOG else "") + ".pdf")
    plt.show()

    # Plot
    bi_idx = (modality_np > 0.2) & (modality_np < 0.8)
    plt.figure(figsize=(8, 6))
    plt.scatter(embedding[bi_idx, 0], embedding[bi_idx, 1], c=modality_np[bi_idx], s=size[bi_idx], cmap='managua', vmin=0, vmax=1)
    plt.colorbar(label='Modality Score')
    plt.title('UMAP of Dictionary Atoms')
    plt.axis('off')
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.tight_layout()
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/umap_bi" + (f"_log" if LOG else "") + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/umap_bi" + (f"_log" if LOG else "") + ".pdf")
    plt.show()


    # Plot
    plt.figure(figsize=(8, 6))
    plt.scatter(embedding[:, 0], embedding[:, 1], c=modality_np, s=size, cmap='managua')
    plt.colorbar(label='Modality Score')
    plt.title('UMAP of Dictionary Atoms')


    ax = plt.gca()

    # Self loops will be drawn as circles
    def curved_edge(p1, p2, curvature=0.2):
        # Midpoint and perpendicular vector for control point
        midpoint = 0.5 * (p1 + p2)
        delta = p2 - p1
        norm = np.linalg.norm(delta)
        if norm == 0:
            return None  # Skip self-loops or overlapping points
        perp = np.array([-delta[1], delta[0]]) / norm
        control = midpoint + curvature * norm * perp

        verts = [p1, control, p2]
        codes = [Path.MOVETO, Path.CURVE3, Path.CURVE3]
        return PathPatch(Path(verts, codes), facecolor='none', edgecolor='gray', lw=1, alpha=0.2)

    # Add curved patches for each bridge above threshold
    for k, (i, j, w) in enumerate(zip(i_idx, j_idx, line_weights)):
        if i == j:
            continue
        patch = curved_edge(embedding[i], embedding[j], curvature=0.15)
        if patch:
            patch.set_linewidth(weights_norm[k])
            ax.add_patch(patch)

    # Add self loops
    def draw_self_loop(ax, center, radius, color, lw, n_points=100):
        theta = np.linspace(0, 2 * np.pi, n_points)
        x = center[0] + radius * np.cos(theta)
        y = center[1] + radius * np.sin(theta) + radius  # upward offset by radius
        ax.plot(x, y, color=color, lw=lw, alpha=0.2, zorder=0)

    # Add self-loops using parametric circle
    for k, (i, j, w) in enumerate(zip(i_idx, j_idx, line_weights)):
        if i != j:
            continue  # Only self-loops

        center = embedding[i]
        node_size = size[i] ** 0.5 / 100  # approximate radius from scatter size
        loop_radius = node_size + weights_norm[k] * 0.5
        loop_thickness = weights_norm[k]

        draw_self_loop(ax, center, loop_radius, color='gray', lw=loop_thickness)

    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/umap_bridge" + (f"_log" if LOG else "") + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/umap_bridge" + (f"_log" if LOG else "") + ".pdf")
    plt.show()

    plt.figure(figsize=(8, 6))
    plt.scatter(embedding_pca[:, 0], embedding_pca[:, 1], c=modality_np, s=size, cmap='managua')
    plt.colorbar(label='Modality Score')
    plt.title('PCA of Dictionary Atoms')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/pca" + (f"_log" if LOG else "") + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/pca" + (f"_log" if LOG else "") + ".pdf")
    plt.show()

    plt.figure(figsize=(8, 6))
    plt.scatter(embedding_weighted_pca[:, 0], embedding_weighted_pca[:, 1], c=modality_np, s=size, cmap='managua')
    plt.colorbar(label='Modality Score')
    plt.title('Weighted PCA of Dictionary Atoms')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/weighted_pca" + (f"_log" if LOG else "") + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/weighted_pca" + (f"_log" if LOG else "") + ".pdf")
    plt.show()

    # visualize an idealized version of "linear separability" :
    # 1. Create dummy data : 2D unit vectors following three Von Mises distributions
    # 2. assign a modality score to each point based on its angle + noise
    # 2.5 assign an energy depending on the cosim with the center of the distribution
    # 3. plot the points colored by modality score

    def von_mises_2d(mu, kappa, size):
        # Generate random angles from a von Mises distribution
        angles = np.random.vonmises(mu, kappa, size)
        # Convert angles to 2D unit vectors
        x = np.cos(angles)
        y = np.sin(angles)
        return np.column_stack((x, y))

    def generate_modality_score(points, noise=0.05):
        # Calculate angles of points
        angles = np.arctan2(points[:, 1], points[:, 0])
        # Normalize angles to [0, 1]
        normalized_angles = (1.5 * angles + np.pi) / (2 * np.pi)
        # Add noise
        noise = np.random.uniform(-noise, noise, size=normalized_angles.shape)
        modality_scores = normalized_angles + noise
        modality_scores = np.clip(modality_scores, 0, 1)
        return modality_scores

    def generate_energy(points, center, noise=5):
        # Calculate angles of points
        cosim = np.dot(points, center) / (np.linalg.norm(points, axis=1) * np.linalg.norm(center))
        # Normalize angles to [0, 1]
        normalized_cosim = (cosim + 1) / 2
        # Add noise
        noise = np.random.uniform(-noise, noise, size=normalized_cosim.shape)
        energy_scores = normalized_cosim + noise
        energy_scores -= energy_scores.min()
        energy_scores /= energy_scores.max()
        energy_scores = np.exp(energy_scores * 2) 
        return energy_scores

    def plot_von_mises(points, energies, modality_scores, title):
        plt.figure(figsize=(8, 8))
        plt.scatter(points[:, 0], points[:, 1], c=modality_scores, cmap='managua', s=energies * 30)
        plt.colorbar(label='Modality Score')
        plt.title(title)
        plt.axis('equal')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/dummy_von_mises.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/dummy_von_mises.pdf")
        plt.show()

    points1 = von_mises_2d(mu=0, kappa=5, size=100)
    energy1 = generate_energy(points1, np.array([1, 0]), noise=0.5)
    points2 = von_mises_2d(mu=2 * np.pi/3, kappa=100, size=100)
    energy2 = generate_energy(points2, np.array([-1, 0]), noise=0.5)
    points3 = von_mises_2d(mu=4 * np.pi/3, kappa=100, size=100)
    energy3 = generate_energy(points3, np.array([-1, 0]), noise=0.5)

    points = np.vstack((points1, points2, points3))
    energies = np.hstack((energy1, energy2, energy3))
    plot_von_mises(points, energies, generate_modality_score(points), "Von Mises Distribution with Modality Score")

In [ ]:
# generate_visualization(SSAE, SSAE_name, SSAE.metrics.mu, SSAE.metrics.E, SSAE.metrics.f, SSAE_bridge["bridge_sigma"])
generate_visualization(ASAE, ASAE_name, ASAE.metrics.mu, ASAE.metrics.E, ASAE.metrics.f, ASAE_bridge["bridge_sigma"])

##### 6.3.1. Features classifying features

#### TODO : projection onto 3D space defined by LR probes (maybe 2D with image & text probes is enough ?)

In [ ]:
eps = modality_threshold
def linear_separability(sae, sae_name, eps=eps):
    D = sae.dictionary._fused_dictionary.cpu()
    modality_score = sae.metrics.mu
    energy_per_concept = sae.metrics.E
    print(D.shape)
    I_mask = (modality_score > 1 - eps).cpu()
    T_mask = (modality_score < eps).cpu()
    B_mask = (modality_score <= 1 - eps).cpu() & (modality_score >= eps).cpu()
    print(f"Number of unimodal concepts: {I_mask.sum()}, {T_mask.sum()}, {B_mask.sum()}")

    mu_weighted_I = (D[I_mask] * energy_per_concept[I_mask].unsqueeze(1).cpu()).sum(dim=0).cpu() / energy_per_concept[I_mask].sum().cpu()
    mu_weighted_T = (D[T_mask] * energy_per_concept[T_mask].unsqueeze(1).cpu()).sum(dim=0).cpu() / energy_per_concept[T_mask].sum().cpu()
    mu_weighted_B = (D[B_mask] * energy_per_concept[B_mask].unsqueeze(1).cpu()).sum(dim=0).cpu() / energy_per_concept[B_mask].sum().cpu()

    d1 = (mu_weighted_B - mu_weighted_I).cpu().numpy()
    d1 /= np.linalg.norm(d1)
    d2 = (mu_weighted_B - mu_weighted_T).cpu().numpy()
    d2 /= np.linalg.norm(d2)

    mu_tout_court = D.mean(dim=0).cpu().numpy()
    mu_weighted_tout_court = (D * energy_per_concept.unsqueeze(1).cpu()).sum(dim=0).cpu().numpy() / energy_per_concept.sum().cpu().numpy()

    D1 = D.cpu().numpy() # shape (N, 512)
    D1 = D1 @ d1.reshape(-1, 1) # shape (N, 1)
    D2 = D.cpu().numpy() # shape (N, 512)
    D2 = D2 @ d2.reshape(-1, 1) # shape (N, 1)
    D_proj = np.concatenate([D1, D2], axis=1) # shape (N, 2)

    modality_color = {
        "bimodal": colorsys.rgb_to_hls(*np.array([91/255, 40/255, 68/255])),
        "image": colorsys.rgb_to_hls(*np.array([128/255, 230/255, 255/255])),
        "text": colorsys.rgb_to_hls(*np.array([255/255, 205/255, 102/255])),
    }
    
    # Normalize energy for plotting size
    energy_np = energy_per_concept.cpu().numpy()
    LOG = False
    if LOG:
        # Log scale energy normalization
        log_energy = np.log(energy_np)  # Log transform with offset to avoid log(0)
        size = log_energy - log_energy.min()
        size = 0.1 * (size) ** 2
    else:
        # Linear scale energy normalization
        size = energy_np - energy_np.min()
        size = 300 * (np.sqrt(size / size.max()))

    def _plot_proj_mu():
        plt.figure(figsize=(8, 6))
        plt.scatter(D_proj[:, 0], D_proj[:, 1], c=modality_score.cpu().numpy(), s=size, cmap='managua')
        plt.colorbar(label='Modality Score')
        plt.axhline(0, color='k', linestyle='--', linewidth=1)
        plt.axvline(0, color='k', linestyle='--', linewidth=1)

        # Plot all the means of the modalities
        plt.scatter(mu_weighted_I.cpu().numpy() @ d1.reshape(-1, 1), mu_weighted_I.cpu().numpy() @ d2.reshape(-1, 1), color=colorsys.hls_to_rgb(modality_color["image"][0], modality_color["image"][1] * 0.3, modality_color["image"][2]), marker='+', s=100, label='mu_I')
        plt.scatter(mu_weighted_T.cpu().numpy() @ d1.reshape(-1, 1), mu_weighted_T.cpu().numpy() @ d2.reshape(-1, 1), color=colorsys.hls_to_rgb(modality_color["text"][0], modality_color["text"][1] * 0.3, modality_color["text"][2]), marker='+', s=100, label='mu_T')
        plt.scatter(mu_weighted_B.cpu().numpy() @ d1.reshape(-1, 1), mu_weighted_B.cpu().numpy() @ d2.reshape(-1, 1), color=colorsys.hls_to_rgb(modality_color["bimodal"][0], modality_color["bimodal"][1] * 0.3, modality_color["bimodal"][2]), marker='+', s=100, label='mu_B')
        plt.scatter(mu_weighted_tout_court @ d1.reshape(-1, 1), mu_weighted_tout_court @ d2.reshape(-1, 1), color='k', marker='+', s=100, label='barycenter')
        plt.xlabel('Projection on mu_B - mu_I')
        plt.ylabel('Projection on mu_B - mu_T')
        plt.title('Projection of D on mu_B - mu_I and mu_B - mu_T')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        os.makedirs(f"./figures/{sae_name.replace('.pt', '')}/geometry", exist_ok=True)
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/projection_mu_B.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/projection_mu_B.pdf")
        plt.show()
    # _plot_proj_mu()

    # Baseline 1 : mu_weighted_{I,T,B}, bias : mu_weighted_tout_court

    def evaluate_custom_probe(weight, bias, trg_mask):
        probe = torch.nn.Linear(D.shape[1], 1, bias=True)
        probe.weight.data = weight
        probe.bias.data = torch.tensor(bias)
        probe.eval()

        with torch.no_grad():
            preds = (D - probe.bias.unsqueeze(0)) @ probe.weight
            
            # print(f"probe_I : {(probe_I > 0).float().mean().item():.4f}, probe_T : {(probe_T > 0).float().mean().item():.4f}, probe_B : {(probe_B > 0).float().mean().item():.4f}")
            acc = (((preds > 0) == trg_mask).float() * energy_per_concept.cpu()).sum() / energy_per_concept.sum()

        return acc.item()
    
    acc_I = evaluate_custom_probe(mu_weighted_I, mu_weighted_tout_court, I_mask)
    acc_T = evaluate_custom_probe(mu_weighted_T, mu_weighted_tout_court, T_mask)
    acc_B = evaluate_custom_probe(mu_weighted_B, mu_weighted_tout_court, B_mask)
    print("Probe 1 : mu_weighted_I", acc_I)
    print("Probe 2 : mu_weighted_T", acc_T)
    print("Probe 3 : mu_weighted_B", acc_B)

    # baseline 2 : fit scikit-learn linear regression for each region :

    def evaluate_probe_sklearn(P, N1, N2, P_mask, N1_mask, N2_mask):
        N = np.concatenate([N1, N2], axis=0)
        N_mask = (N1_mask | N2_mask).cpu().numpy()
        X = np.concatenate([P, N], axis=0)
        y = np.concatenate([np.ones(P.shape[0]), np.zeros(N.shape[0])], axis=0)
        
        # Shuffle
        perm = np.random.permutation(X.shape[0])
        X, y = X[perm], y[perm]
        energy_perm = energy_per_concept.cpu().numpy()[perm]
        P_mask = P_mask.cpu().numpy()[perm]
        N_mask = N_mask[perm]
        
        # Compute sample weights
        energy_perm[P_mask] = energy_perm[P_mask] / energy_perm[P_mask].sum() / 2
        energy_perm[N_mask] = energy_perm[N_mask] / energy_perm[N_mask].sum() / 2
        
        model = LinearRegression()
        model.fit(X, y, sample_weight=energy_perm)
        y_pred = model.predict(X)
        y_pred = (y_pred > 0.5).astype(int)
        y_pred = (y_pred == y).astype(int)
        return (y_pred * energy_perm).sum() / energy_perm.sum(), model
    
    acc_lr_B, lr_probe_B = evaluate_probe_sklearn(D[B_mask], D[I_mask], D[T_mask], B_mask, I_mask, T_mask)
    acc_lr_I, lr_probe_I = evaluate_probe_sklearn(D[I_mask], D[B_mask], D[T_mask], I_mask, B_mask, T_mask)
    acc_lr_T, lr_probe_T = evaluate_probe_sklearn(D[T_mask], D[I_mask], D[B_mask], T_mask, I_mask, B_mask)
    print("Probe 4 : Linear Regression B", acc_lr_B)
    print("Probe 5 : Linear Regression I", acc_lr_I)
    print("Probe 6 : Linear Regression T", acc_lr_T)
    
    # Plot projection on probe directions :
    lr_d_B = lr_probe_B.coef_.reshape(-1)
    d1 = lr_d_B / np.linalg.norm(lr_d_B)
    lr_d_I = lr_probe_I.coef_.reshape(-1)
    d2 = lr_d_I / np.linalg.norm(lr_d_I)
    lr_d_T = lr_probe_T.coef_.reshape(-1)
    d3 = lr_d_T / np.linalg.norm(lr_d_T)
    
    D_ = D.cpu().numpy()
    x1 = D_ @ d1.reshape(-1, 1)  # projection on I
    x2 = D_ @ d2.reshape(-1, 1)  # projection on T
    x3 = D_ @ d3.reshape(-1, 1)  # projection on B*
    
    from mpl_toolkits.mplot3d import Axes3D
    import plotly.graph_objects as go
    
    def mpl_to_plotly(cmap_name, pl_entries=255, luminance_scale=1.0):
        cmap = plt.cm.get_cmap(cmap_name, pl_entries)
        h = 1.0 / (pl_entries - 1)
        colorscale = []

        for k in range(pl_entries):
            r, g, b, _ = cmap(k)
            hls = colorsys.rgb_to_hls(r, g, b)
            l_scaled = max(0.0, min(1.0, hls[1] * luminance_scale))
            r_new, g_new, b_new = colorsys.hls_to_rgb(hls[0], l_scaled, hls[2])
            colorscale.append([
                round(k * h, 4),
                f'rgb({int(r_new * 255)}, {int(g_new * 255)}, {int(b_new * 255)})'
            ])

        return colorscale
    
    managua_plotly = mpl_to_plotly('managua')
    dark_managua_plotly = mpl_to_plotly('managua', luminance_scale=0.5)

    class_score = modality_score.cpu().numpy()
    
    fig = go.Figure(data=[
        go.Scatter3d(
            x=x1.squeeze(), y=x2.squeeze(), z=x3.squeeze(),
            mode='markers',
            marker=dict(
                size=np.power(size**2, 1/3) * 0.5,
                color=class_score,
                colorscale=managua_plotly,  # or 'managua' if you have a custom Plotly scale
                colorbar=dict(title='Modality Score'),
                opacity=0.8,
                line=dict(width=0),
            )
        )
    ])
    
    fig.add_trace(go.Scatter3d(
            x=x1.squeeze(), y=x2.squeeze(), z=np.zeros_like(x3.squeeze()) + min(x3),
            mode='markers',
            marker=dict(
                size=np.power(size**2, 1/3) * 0.5,
                color=class_score,
                colorscale=dark_managua_plotly,  # or 'managua' if you have a custom Plotly scale
                # colorbar=dict(title='Modality Score'),
                opacity=0.1,
                line=dict(width=0),
                symbol='circle',
            )
        )
    )

    def add_arrow(fig, origin, direction, color, name):
        # Normalize and scale arrow length
        norm_dir = direction / np.linalg.norm(direction)
        length = np.linalg.norm(direction)
        end = origin + norm_dir * length
        # if color is a tuple of numpy arrays, convert it to a string RGB format
        if isinstance(color, tuple) and all(isinstance(c, np.ndarray) for c in color):
            color = f'rgb({int(color[0] * 255)}, {int(color[1] * 255)}, {int(color[2] * 255)})'
        fig.add_trace(go.Scatter3d(
            x=[origin[0], end[0]],
            y=[origin[1], end[1]],
            z=[origin[2], end[2]],
            mode='lines+markers',
            name=name,
            line=dict(color=[color, color], width=6),
            marker=dict(size=5, color=[color, color], symbol='circle')
        ))
    
    # Get mean vectors projected into the 3D space
    mu_B = (mu_weighted_B.cpu().numpy() @ np.stack([d1, d2, d3], axis=1)).flatten()
    mu_I = (mu_weighted_I.cpu().numpy() @ np.stack([d1, d2, d3], axis=1)).flatten()
    mu_T = (mu_weighted_T.cpu().numpy() @ np.stack([d1, d2, d3], axis=1)).flatten()


    # Add arrows from origin to means
    add_arrow(fig, origin=np.zeros(3), direction=mu_I, color=colorsys.hls_to_rgb(modality_color["image"][0], modality_color["image"][1] * 0.3, modality_color["image"][2]), name='μ_I')
    add_arrow(fig, origin=np.zeros(3), direction=mu_T, color=colorsys.hls_to_rgb(modality_color["text"][0], modality_color["text"][1] * 0.3, modality_color["text"][2]), name='μ_T')
    add_arrow(fig, origin=np.zeros(3), direction=mu_B, color=colorsys.hls_to_rgb(modality_color["bimodal"][0], modality_color["bimodal"][1] * 0.3, modality_color["bimodal"][2]), name='μ_B')
    add_arrow(fig, origin=np.zeros(3), direction=mu_weighted_tout_court @ np.stack([d1, d2, d3], axis=1), color='black', name='μ')

    fig.update_layout(
        scene=dict(
            xaxis=dict(
                backgroundcolor='rgba(0,0,0,0)',
                gridcolor='rgba(0,0,0,0)',
                zeroline=False,
                showbackground=False
            ),
            yaxis=dict(
                backgroundcolor='rgba(0,0,0,0)',
                gridcolor='rgba(0,0,0,0)',
                zeroline=False,
                showbackground=False
            ),
            zaxis=dict(
                backgroundcolor='rgba(0.9,0.9,0.9,1)',
                # gridcolor='rgba(1,1,1,1)',
                # zeroline=False,
                # showbackground=False
            ),
            xaxis_title=r'LR_B',
            yaxis_title=r'LR_I',
            zaxis_title=r'LR_T',
        ),
        margin=dict(l=0, r=0, b=0, t=30),
        title="3D Projection on Linear Regression Directions",
        scene_camera=dict(
            eye=dict(x=1., y=2., z=0.6),
        )
    )
    # save html and pdf fig :
    fig.write_html(f"./figures/{sae_name.replace('.pt', '')}/geometry/projection_lr.html")
    fig.write_image(f"./figures/{sae_name.replace('.pt', '')}/geometry/projection_lr.pdf")
    fig.write_image(f"./figures/{sae_name.replace('.pt', '')}/geometry/projection_lr.png", scale=5)
    fig.show()
    
    # Now, consider each D_i as a probe and measure it's accuracy :

    accs = []
    for i in range(D.shape[0]):
        probe = D[i]
        if B_mask[i]:
            trg_mask = B_mask
        elif I_mask[i]:
            trg_mask = I_mask
        elif T_mask[i]:
            trg_mask = T_mask
        else:
            raise ValueError("Unknown modality")
        acc = evaluate_custom_probe(probe, mu_weighted_tout_court, trg_mask)
        accs.append(acc)

    print(f"Mean acc : {np.mean(accs):.4f}, std : {np.std(accs):.4f}")
    e = energy_per_concept.cpu().numpy() / energy_per_concept.sum().item()
    print(f"Mean acc : {(np.array(accs) * e).sum():.4f}")

    def _plot_acc_hist():
        plt.figure(figsize=(10, 6))
        plt.hist(accs, bins=100, range=(0, 1), color='blue', label='Probe Accuracy', weights=energy_per_concept.cpu().numpy() / energy_per_concept.sum().item())
        plt.title("Distribution of Probe Accuracy")
        plt.xlabel("Probe Accuracy")
        plt.ylabel("density")
        #plt.yscale("log")
        plt.grid(axis='y', alpha=0.75)
        plt.legend()
        # Add mean and std information to the plot
        # plt.text(0.05, 0.95, f"Mean: {np.mean(accs):.4f}\nStd: {np.std(accs):.4f}", transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/probe_accuracy.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/probe_accuracy.pdf")
        plt.show()

        # plot accuracy vs modality score :
        plt.figure(figsize=(10, 6))
        plt.hexbin(modality_score.cpu().numpy(), accs, gridsize=30, cmap='magma_r', bins='log', C=energy_per_concept.cpu().numpy(), reduce_C_function=np.sum, norm=LogNorm(vmin=max(1e-4, energy_per_concept.cpu().numpy().min())))
        plt.colorbar(label='density', extend='min')
        plt.title("Probe Accuracy vs Modality Score")
        plt.xlabel("Modality Score")
        plt.ylabel("Probe Accuracy")
        plt.grid(axis='y', alpha=0.75)
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/probe_accuracy_vs_modality_score.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/probe_accuracy_vs_modality_score.pdf")
        plt.show()
    _plot_acc_hist()

    sae.metrics._add_more({
        'linear_separability': {
            'acc_I': acc_I,
            'acc_T': acc_T,
            'acc_B': acc_B,
            'acc_lr_B': acc_lr_B,
            'acc_lr_I': acc_lr_I,
            'acc_lr_T': acc_lr_T,
            'feature_probe': (np.array(accs) * energy_per_concept.cpu().numpy()).sum() / energy_per_concept.sum(),
        }
    }, save=True)
    
    def _plot_probe_accuracy_bar_colored_by_modality():
        plt.figure(figsize=(10, 6))
        cmap = plt.get_cmap("managua")
        norm = plt.Normalize(vmin=0, vmax=1)
        bins = np.linspace(0, 1, 101)  # 30 bins
        bin_centers = 0.5 * (bins[:-1] + bins[1:])  # Bin centers for plotting
        digitized = np.digitize(accs, bins) - 1
        weights = energy_per_concept.cpu().numpy() / energy_per_concept.sum().item()
        plt.figure(figsize=(10, 6))
        for b in range(len(bins) - 1):
            b_idx = np.where(digitized == b)[0]
            if len(b_idx) == 0:
                continue
            sorted_idx = b_idx[np.argsort(modality_score[b_idx].cpu().numpy())]
            bottom = 0
            for i in sorted_idx:
                height = weights[i]
                color = cmap(norm(modality_score[i].cpu().numpy()))  # modality -> color
                plt.bar(bin_centers[b], height, width=(bins[1] - bins[0]), bottom=bottom, color=color, edgecolor='none')
                bottom += height
        sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
        sm.set_array([])  # Dummy array for ScalarMappable
        cbar = plt.colorbar(sm, ax=plt.gca(), orientation='vertical', label='Modality Score') 
        plt.title("Distribution of Probe Accuracy")
        plt.xlabel("Probe Accuracy")
        plt.ylabel("density")
        plt.grid(axis='y', alpha=0.75)
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/probe_accuracy_bar_colored_by_modality.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/probe_accuracy_bar_colored_by_modality.pdf")
        plt.show()
    _plot_probe_accuracy_bar_colored_by_modality()


In [ ]:
linear_separability(SSAE, SSAE_name, eps=eps)

In [ ]:
linear_separability(ASAE, ASAE_name, eps=eps)

In [ ]:
print(SSAE.metrics)
print(ASAE.metrics)

##### 6.3.2. Features classifying Data

In [ ]:
eps = modality_threshold # Unimodality threshold
@torch.no_grad()
def data_classifiability(sae, sae_name, eps=eps, skip_plot=False):
    I = train_loader.dataset.tensors[0]
    I = I[torch.randperm(I.shape[0])[:int(1e4)]]
    T = train_loader.dataset.tensors[1]
    T = T[torch.randperm(T.shape[0])[:int(1e4)]]
    D_ata = torch.cat([I, T], dim=0) # shape (N, 512)
    D_ict = sae.dictionary._fused_dictionary.cpu() # shape (M, 512)
    modality_score = sae.metrics.mu
    energy_per_concept = sae.metrics.E
    print(D_ict.shape)
    I_mask = (modality_score > 1 - eps).cpu()
    T_mask = (modality_score < eps).cpu()
    B_mask = (modality_score <= 1 - eps).cpu() & (modality_score >= eps).cpu()
    print(f"Number of unimodal concepts: {I_mask.sum()}, {T_mask.sum()}, {B_mask.sum()}")

    mu_weighted_I = (D_ict[I_mask] * energy_per_concept[I_mask].unsqueeze(1).cpu()).sum(dim=0).cpu() / energy_per_concept[I_mask].sum().cpu()
    mu_weighted_T = (D_ict[T_mask] * energy_per_concept[T_mask].unsqueeze(1).cpu()).sum(dim=0).cpu() / energy_per_concept[T_mask].sum().cpu()
    mu_weighted_B = (D_ict[B_mask] * energy_per_concept[B_mask].unsqueeze(1).cpu()).sum(dim=0).cpu() / energy_per_concept[B_mask].sum().cpu()

    mu_weighted_tout_court = (D_ict * energy_per_concept.unsqueeze(1).cpu()).sum(dim=0).cpu().numpy() / energy_per_concept.sum().cpu().numpy()

    # Baseline 1 : mu_weighted_{I,T,B}, bias : mu_weighted_tout_court

    def evaluate_custom_probe(weight, bias, trg_mask):
        probe = torch.nn.Linear(D_ict.shape[1], 1, bias=True)
        probe.weight.data = weight
        probe.bias.data = torch.tensor(bias)
        probe.eval()

        with torch.no_grad():
            preds = (D_ata - probe.bias.unsqueeze(0)) @ probe.weight
            
            # print(f"probe_I : {(probe_I > 0).float().mean().item():.4f}, probe_T : {(probe_T > 0).float().mean().item():.4f}, probe_B : {(probe_B > 0).float().mean().item():.4f}")
            acc = ((preds > 0) == trg_mask).float().mean()

        return acc.item()
    
    Image_Data_Mask = torch.zeros(D_ata.shape[0], dtype=torch.bool)
    Image_Data_Mask[:I.shape[0]] = True
    Image_Data_Mask = Image_Data_Mask.cpu().numpy()
    Text_Data_Mask = torch.zeros(D_ata.shape[0], dtype=torch.bool)
    Text_Data_Mask[I.shape[0]:] = True
    Text_Data_Mask = Text_Data_Mask.cpu().numpy()
    acc_I = evaluate_custom_probe(mu_weighted_I, mu_weighted_tout_court, Image_Data_Mask) # Target accuracy : 1
    acc_T = evaluate_custom_probe(mu_weighted_T, mu_weighted_tout_court, Text_Data_Mask) # Target accuracy : 1
    acc_B = evaluate_custom_probe(mu_weighted_B, mu_weighted_tout_court, Image_Data_Mask) # Target accuracy : 0.5
    acc_B = max(acc_B, 1-acc_B)
    print("Probe 1 : mu_weighted_I", acc_I)
    print("Probe 2 : mu_weighted_T", acc_T)
    print("Probe 3 : mu_weighted_B", acc_B)

    # baseline 2 : fit scikit-learn linear regression for each region :

    def evaluate_probe_sklearn(P, N1, N2, P_mask, N1_mask, N2_mask):
        N = np.concatenate([N1, N2], axis=0)
        N_mask = (N1_mask | N2_mask).cpu().numpy()
        X = np.concatenate([P, N], axis=0)
        y = np.concatenate([np.ones(P.shape[0]), np.zeros(N.shape[0])], axis=0)
        
        # Shuffle
        perm = np.random.permutation(X.shape[0])
        X, y = X[perm], y[perm]
        energy_perm = energy_per_concept.cpu().numpy()[perm]
        P_mask = P_mask.cpu().numpy()[perm]
        N_mask = N_mask[perm]
        
        # Compute sample weights
        energy_perm[P_mask] = energy_perm[P_mask] / energy_perm[P_mask].sum() / 2
        energy_perm[N_mask] = energy_perm[N_mask] / energy_perm[N_mask].sum() / 2
        
        model = LinearRegression()
        model.fit(X, y, sample_weight=energy_perm)
        y_pred = model.predict(D_ata.cpu().numpy())
        y_pred = (y_pred > 0.5).astype(int)
        y_pred = (y_pred == Image_Data_Mask).astype(int)
        return y_pred.mean()
    
    acc_lr_B = evaluate_probe_sklearn(D_ict[B_mask], D_ict[I_mask], D_ict[T_mask], B_mask, I_mask, T_mask) # Target accuracy : 0.5
    acc_lr_B = max(acc_lr_B, 1-acc_lr_B)
    acc_lr_I = evaluate_probe_sklearn(D_ict[I_mask], D_ict[B_mask], D_ict[T_mask], I_mask, B_mask, T_mask) # Target accuracy : 1
    acc_lr_T = evaluate_probe_sklearn(D_ict[T_mask], D_ict[I_mask], D_ict[B_mask], T_mask, I_mask, B_mask) # Target accuracy : 0
    acc_lr_T = 1 - acc_lr_T # because we want to predict the text data
    print("Probe 4 : Linear Regression B", acc_lr_B)
    print("Probe 5 : Linear Regression I", acc_lr_I)
    print("Probe 6 : Linear Regression T", acc_lr_T)

    # Now, consider each D_i as a probe and measure it's accuracy :

    accs = []
    accs_trg = []
    for i in tqdm(range(D_ict.shape[0])):
        probe = D_ict[i]
        if B_mask[i]:
            trg_mask = Image_Data_Mask
        elif I_mask[i]:
            trg_mask = Image_Data_Mask
        elif T_mask[i]:
            trg_mask = Text_Data_Mask
        else:
            raise ValueError("Unknown modality")
        acc = evaluate_custom_probe(probe, mu_weighted_tout_court, trg_mask)
        if B_mask[i]:
            acc = max(acc, 1 - acc)
            accs.append(acc)
            accs_trg.append(1.5 - acc)
        else:
            accs.append(acc)
            accs_trg.append(acc)

    print(f"Mean unweighted acc : {np.mean(accs):.4f}, std : {np.std(accs):.4f}")
    print(f"Mean acc : {(np.array(accs_trg) * energy_per_concept.cpu().numpy()).sum() / energy_per_concept.sum():.4f}")

    if not skip_plot:
        plt.figure(figsize=(10, 6))
        plt.hist(accs, bins=100, range=(0, 1), color='blue', label='Probe Accuracy', weights=energy_per_concept.cpu().numpy() / energy_per_concept.sum().item())
        plt.title("Distribution of Probe Accuracy")
        plt.xlabel("Probe Accuracy")
        plt.ylabel("density")
        #plt.yscale("log")
        plt.grid(axis='y', alpha=0.75)
        plt.legend()
        # Add mean and std information to the plot
        # plt.text(0.05, 0.95, f"Mean: {np.mean(accs):.4f}\nStd: {np.std(accs):.4f}", transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/data_probe_accuracy.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/data_probe_accuracy.pdf")
        plt.show()

        # plot accuracy vs modality score :
        plt.figure(figsize=(10, 6))
        plt.hexbin(modality_score.cpu().numpy(), accs, gridsize=30, cmap='magma_r', bins='log', C=energy_per_concept.cpu().numpy(), reduce_C_function=np.sum, norm=LogNorm(vmin=max(1e-4, energy_per_concept.cpu().numpy().min())))
        plt.colorbar(label='density', extend='min')
        plt.title("Probe Accuracy vs Modality Score")
        plt.xlabel("Modality Score")
        plt.ylabel("Probe Accuracy")
        plt.grid(axis='y', alpha=0.75)
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/data_probe_accuracy_vs_modality_score.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/data_probe_accuracy_vs_modality_score.pdf")
        plt.show()

        modality_perm = torch.argsort(modality_score).cpu()

        modality_sorted = modality_score[modality_perm]
        accs_sorted = np.array(accs)[modality_perm.cpu().numpy()]

        values = modality_sorted.cpu().numpy()
        weights = (energy_per_concept[modality_perm].cpu().numpy())
        weights = weights / weights.sum()

        bins = np.linspace(0, 1, 31)
        digitized = np.digitize(values, bins) - 1 # shape (N,), where N is the number of concepts. Each value is the index of the bin it belongs to.
        digitized = np.clip(digitized, 0, len(bins) - 2)

        # Weighted mean per bin
        numerator = np.bincount(digitized, weights=accs_sorted * weights, minlength=len(bins)-1)
        denominator = np.bincount(digitized, weights=weights, minlength=len(bins)-1)
        mean_per_bin = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator != 0)
        for b in range(len(mean_per_bin)):
            b_idx = np.where(digitized == b)[0]
            if len(b_idx) == 0:
                mean_per_bin[b] = 0
            else:
                mean_per_bin[b] = np.sum(accs_sorted[b_idx] * weights[b_idx]) / np.sum(weights[b_idx]) if np.sum(weights[b_idx]) > 0 else 0

        bin_centers = 0.5 * (bins[:-1] + bins[1:])
        plt.figure(figsize=(10, 6))
        plt.plot(bin_centers, mean_per_bin, color='blue')
        plt.xlabel("Modality Score")
        plt.ylabel("Weighted Mean per Bin")
        plt.title("Histogram of Modality Score Means (Weighted)")
        plt.grid(axis='y', alpha=0.75)
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/data_probe_accuracy_vs_modality_score.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/data_probe_accuracy_vs_modality_score.pdf")
        plt.show()

        plt.figure(figsize=(10, 6))
        cmap = plt.get_cmap("managua")
        norm = plt.Normalize(vmin=0, vmax=1)
        bins = np.linspace(0, 1, 101)  # 30 bins
        bin_centers = 0.5 * (bins[:-1] + bins[1:])  # Bin centers for plotting
        digitized = np.digitize(accs_sorted, bins) - 1
        plt.figure(figsize=(10, 6))
        for b in range(len(bins) - 1):
            b_idx = np.where(digitized == b)[0]
            if len(b_idx) == 0:
                continue
            # Sort i_j by energy
            sorted_idx = b_idx[np.argsort(values[b_idx])]
            bottom = 0
            for i in sorted_idx:
                height = weights[i]
                color = cmap(norm(values[i]))  # modality -> color
                plt.bar(bin_centers[b], height, width=(bins[1] - bins[0]), bottom=bottom, color=color, edgecolor='none')
                bottom += height
        sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
        sm.set_array([])  # Dummy array for ScalarMappable
        cbar = plt.colorbar(sm, ax=plt.gca(), orientation='vertical', label='Modality Score') 
        plt.title("Distribution of Probe Accuracy")
        plt.xlabel("Probe Accuracy")
        plt.ylabel("density")
        plt.grid(axis='y', alpha=0.75)
        plt.ylim(0, 0.2)
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/data_probe_accuracy_bar_colored_by_modality.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"./figures/{sae_name.replace('.pt', '')}/geometry/data_probe_accuracy_bar_colored_by_modality.pdf")
        plt.show()
        
    sae.metrics._add_more({
        'data_cassifiability': {
            'acc_I': acc_I,
            'acc_T': acc_T,
            'acc_B': acc_B,
            'acc_lr_B': acc_lr_B,
            'acc_lr_I': acc_lr_I,
            'acc_lr_T': acc_lr_T,
            'feature_probe': (np.array(accs_trg) * energy_per_concept.cpu().numpy()).sum() / energy_per_concept.sum(),
        }
    }, save=True)


In [ ]:
data_classifiability(SSAE, SSAE_name, eps=eps)

In [ ]:
data_classifiability(ASAE, ASAE_name, eps=eps)

##### 6.3.3. Dictionary orthogonality : $D_I \bot D_T \bot D_B$

In [ ]:
def get_dict_orthogonality(sae, modality_score):
    D = sae.dictionary._fused_dictionary.cpu().numpy()
    D = D / np.linalg.norm(D, axis=1, keepdims=True)  # Normalize each vector to unit length
    orthogonality = (np.dot(D, D.T))
    I_mask = (modality_score > 0.9).cpu().numpy()
    T_mask = (modality_score < 0.1).cpu().numpy()
    B_mask = (modality_score <= 0.9).cpu().numpy() & (modality_score >= 0.1).cpu().numpy()
    sae.metrics._add_more({
        'orthogonality': {
            'I_ortho_T': orthogonality[I_mask][:, T_mask].mean(),
            'I_ortho_B': orthogonality[I_mask][:, B_mask].mean(),
            'T_ortho_B': orthogonality[T_mask][:, B_mask].mean(),
            'I_ortho_I': orthogonality[I_mask][:, I_mask].mean(),
            'T_ortho_T': orthogonality[T_mask][:, T_mask].mean(),
            'B_ortho_B': orthogonality[B_mask][:, B_mask].mean(),
        }
    }, save=True)

In [ ]:
get_dict_orthogonality(SSAE, SSAE.metrics.mu)
get_dict_orthogonality(ASAE, ASAE.metrics.mu)

#### 6.4. Modality Gap & Contrastive Loss

In [ ]:
@torch.no_grad()
def get_modality_gap(I, T, sae, data_name="default"):
    # Measure the modality gap :
    # - norm of difference in mean image and text embeddings
    # - wasserstein distance between the image and text embeddings distributions.
    N_subsample = int(1e4) # dataset has 1e6 images and texts, subsample to 1e4 for speed.
    # raise
    # # TODO : try modality gap wasserstein without 
    p1 = torch.randperm(I.shape[0])[:N_subsample]
    p2 = torch.randperm(T.shape[0])[:N_subsample]
    T = I[p2].cpu()
    I = I[p1].cpu() # shape (N_subsample, D)
    I_mean = I.mean(axis=0) # shape (D,)
    T_mean = T.mean(axis=0)
    DiM = np.linalg.norm(I_mean - T_mean)
    c = metrics.Wasserstein(I, T, metric='cosim')
    sae.metrics._add_more({
        f"modality_gap_{data_name}": {
            "DiM": DiM,
            "Wasserstein": c,
        }
    }, save=True)

def get_contrastive_loss(I, T, sae, data_name="default"):
    # Track Contrastive Loss & accuracy for Image-Text pairs
    # TODO : retrain the logit_scale parameter to fit the data
    batch_size = 256
    num_batches = I.shape[0] // batch_size

    class LogitScale(torch.nn.Module):
        def __init__(self, initial_value=4.6052):
            super(LogitScale, self).__init__()
            self.logit_scale = torch.nn.Parameter(torch.tensor(initial_value))

        def forward(self, x):
            return torch.exp(self.logit_scale) * x
    logit_scale = LogitScale()
    optimizer = torch.optim.Adam(logit_scale.parameters(), lr=1e-3)

    for ep in range(1, 3):
        contrastive_loss = 0.0
        correct_pairs = 0
        total_pairs = 0
        p = 0
        for i in tqdm(range(num_batches)):
            I_batch = I[i * batch_size:(i + 1) * batch_size]
            T_batch = T[i * batch_size:(i + 1) * batch_size]

            # Compute cosine similarity
            I_batch = I_batch / (I_batch.norm(dim=1, keepdim=True) + 1e-8)
            T_batch = T_batch / (T_batch.norm(dim=1, keepdim=True) + 1e-8)
            logits_per_image = I_batch @ T_batch.T
            logits_per_text = T_batch @ I_batch.T

            # logit_scale = torch.exp(torch.tensor(4.6052)) # from CLIP ViT-B/32

            logits_per_image = logit_scale(logits_per_image)
            logits_per_text = logit_scale(logits_per_text)

            labels = torch.arange(batch_size, device=I_batch.device)
            loss_image = F.cross_entropy(logits_per_image, labels)
            loss_text = F.cross_entropy(logits_per_text, labels)
            loss = (loss_image + loss_text) / 2
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            contrastive_loss += loss.item()

            acc_image = (logits_per_image.argmax(dim=1) == labels).sum().item()
            acc_text = (logits_per_text.argmax(dim=1) == labels).sum().item()
            correct_pairs += (acc_image + acc_text) / 2
            total_pairs += batch_size

            # p : probability of correct labels
            p_image = torch.softmax(logits_per_image, dim=1)
            p_image = p_image[torch.arange(batch_size, device=I_batch.device), labels].mean().item()
            p_text = torch.softmax(logits_per_text, dim=1)
            p_text = p_text[torch.arange(batch_size, device=I_batch.device), labels].mean().item()
            p += (p_image + p_text) / 2
        contrastive_loss /= num_batches
        accuracy = correct_pairs / total_pairs
        print(f"Contrastive Loss: {contrastive_loss:.4f}, Accuracy: {accuracy:.4f}, Probability: {p / num_batches:.4f}")
    sae.metrics._add_more({
        f"contrastive_loss_{data_name}": {
            "loss": contrastive_loss,
            "accuracy": accuracy,
            "probability": p / num_batches,
        }
    }, save=True)

In [ ]:
get_modality_gap(train_loader.dataset.tensors[0], train_loader.dataset.tensors[1], SSAE)
get_modality_gap(train_loader.dataset.tensors[0], train_loader.dataset.tensors[1], ASAE)

get_contrastive_loss(train_loader.dataset.tensors[0], train_loader.dataset.tensors[1], SSAE)
get_contrastive_loss(train_loader.dataset.tensors[0], train_loader.dataset.tensors[1], ASAE)

In [ ]:
get_modality_gap(I_bi_S, T_bi_S, SSAE, data_name="bimodal")
get_modality_gap(I_bi_A, T_bi_A, ASAE, data_name="bimodal")

get_contrastive_loss(I_bi_S, T_bi_S, SSAE, data_name="bimodal")
get_contrastive_loss(I_bi_A, T_bi_A, ASAE, data_name="bimodal")

I_ae_S, T_ae_S, B_ae_S = get_bimodal_data(SSAE, train_loader, SSAE.metrics.mu, device, eps=0.0)
I_ae_A, T_ae_A, B_ae_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, eps=0.0)

# plot_cosim_histogram(I_ae_S, T_ae_S, B_ae_S, SSAE_name, subtitle="AE")
# plot_cosim_histogram(I_ae_A, T_ae_A, B_ae_A, ASAE_name, subtitle="AE")
# fit_and_plot_pca(I_ae_S, T_ae_S, B_ae_S, SSAE_name, save_title='data_ae_pca_pc1_pc2')
# fit_and_plot_pca(I_ae_A, T_ae_A, B_ae_A, ASAE_name, save_title='data_ae_pca_pc1_pc2')
get_modality_gap(I_ae_S, T_ae_S, SSAE, data_name="AE")
get_modality_gap(I_ae_A, T_ae_A, ASAE, data_name="AE")
get_contrastive_loss(I_ae_S, T_ae_S, SSAE, data_name="AE")
get_contrastive_loss(I_ae_A, T_ae_A, ASAE, data_name="AE")

In [ ]:
print("SSAE metrics:")
print(SSAE.metrics)
print("ASAE metrics:")
print(ASAE.metrics)

In [ ]:
I_uni_S, T_uni_S, B_uni_S = get_bimodal_data(SSAE, train_loader, SSAE.metrics.mu, device, complement=True)
I_uni_A, T_uni_A, B_uni_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True)
get_modality_gap(I_uni_S, T_uni_S, SSAE, data_name="unimodal")
get_modality_gap(I_uni_A, T_uni_A, ASAE, data_name="unimodal")

get_contrastive_loss(I_uni_S, T_uni_S, SSAE, data_name="unimodal")
get_contrastive_loss(I_uni_A, T_uni_A, ASAE, data_name="unimodal")


In [ ]:
I_bridge_S, T_bridge_S, B_bridge_S = get_bimodal_data(SSAE, train_loader, SSAE.metrics.mu, device, complement=False, use_E_B=True, tau=0.05)
I_bridge_A, T_bridge_A, B_bridge_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=False, use_E_B=True, tau=0.05)
get_modality_gap(I_bridge_S, T_bridge_S, SSAE, data_name="bridge")
get_modality_gap(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")

get_contrastive_loss(I_bridge_S, T_bridge_S, SSAE, data_name="bridge")
get_contrastive_loss(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")

In [ ]:
I_bridge_C_S, T_bridge_C_S, B_bridge_C_S = get_bimodal_data(SSAE, train_loader, SSAE.metrics.mu, device, complement=True, use_E_B=True, tau=0.31)
I_bridge_C_A, T_bridge_C_A, B_bridge_C_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True, use_E_B=True, tau=0.31)
get_modality_gap(I_bridge_C_S, T_bridge_C_S, SSAE, data_name="bridge_C")
get_modality_gap(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")

get_contrastive_loss(I_bridge_C_S, T_bridge_C_S, SSAE, data_name="bridge_C")
get_contrastive_loss(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")

In [ ]:
print(SSAE.metrics)
print(ASAE.metrics)

# torch.save(SSAE.metrics, f"./figures/{SSAE_name.replace('.pt', '')}/metrics.pt")
# torch.save(ASAE.metrics, f"./figures/{ASAE_name.replace('.pt', '')}/metrics.pt")

In [ ]:
best = None
best_bi_acc = 0
for i in range(10):
    ASAE, ASAE_name = __train(beta=5e-4, force_retrain=True, save_quand_meme=True) # Aligned SAE
    ASAE.metrics = measure_everything(ASAE, train_loader, device, return_sqr=True)
    ASAE_bridge = {
        "bridge_sigma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False).cpu().detach(),
        # "bridge_sigma_null_C" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=True, null_D=False).cpu().detach(),
        # "bridge_sigma_null_D" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=True).cpu().detach(),
        "bridge_sigma_null" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=True, null_D=True).cpu().detach(),
    }
    _prune_bridge(ASAE_bridge["bridge_sigma"], ASAE_bridge["bridge_sigma_null"])
    get_bridge_norms(ASAE_bridge["bridge_sigma"], ASAE.metrics.mu, ASAE.metrics.E, ASAE_name, ASAE)

    I_bi_A, T_bi_A, B_bi_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device)
    I_uni_A, T_uni_A, B_uni_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True)
    I_bridge_A, T_bridge_A, B_bridge_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=False, use_E_B=True)
    I_bridge_C_A, T_bridge_C_A, B_bridge_C_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True, use_E_B=True)

    # get_modality_gap(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    # get_modality_gap(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    # get_modality_gap(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    # get_modality_gap(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    get_contrastive_loss(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    get_contrastive_loss(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    get_contrastive_loss(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    get_contrastive_loss(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    print(ASAE.metrics)
    bi_acc = ASAE.metrics.contrastive_loss_bimodal['accuracy']
    uni_acc = ASAE.metrics.contrastive_loss_unimodal['accuracy']
    bridge_acc = ASAE.metrics.contrastive_loss_bridge['accuracy']
    bridge_C_acc = ASAE.metrics.contrastive_loss_bridge_C['accuracy']
    print(f"Iteration {i+1} - Bimodal Acc: {bi_acc:.4f}, Unimodal Acc: {uni_acc:.4f}, Bridge Acc: {bridge_acc:.4f}, Bridge C Acc: {bridge_C_acc:.4f}")
    if bi_acc > best_bi_acc:
        best_bi_acc = bi_acc
        best = ASAE
        print(f"New best model found with Bimodal Acc: {best_bi_acc:.4f}")

In [ ]:
for sae, m in zip(all_saes, all_metrics):
    print(m)

In [ ]:
# BatchTopK, MP
all_metrics = []
all_saes = []
for useTopK in [False]:#, False]:
    print(f"Training with useTopK={useTopK}")
    top_k = 50
    expansion = 64
    lr = 1e-4
    epochs = 25
    ASAE, ASAE_name, logs = __train(beta=0e-4, force_retrain=True, save_quand_meme=True, UseTOPK=useTopK, top_k=top_k, expansion_factor=expansion, epochs=epochs, lr=lr) # Aligned SAE
    ASAE.metrics = measure_everything(ASAE, train_loader, device, return_sqr=True)
    print(f"Proportion of energy in bimodal concepts: {ASAE.metrics.E_in_B:.4f}")
    ASAE_bridge = {
        "bridge_sigma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False).cpu().detach(),
        "bridge_gamma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False, weight_type="OT").cpu().detach(),
    }
    
    get_sparse_reconstruction_metrics(ASAE, train_loader)
    get_C_metrics(ASAE, train_loader, top_k=top_k)

    # SSAE.metrics._add_more(get_stability_beta(train_loader, beta=0e-4), save=True)
    # ASAE.metrics._add_more(get_stability_beta(train_loader, beta=5e-4), save=True)

    get_D_structure_metrics(ASAE)
    # get_Z_structure_metrics(ASAE, train_loader)
    
    get_bridge_norms(ASAE_bridge["bridge_sigma"], ASAE.metrics.mu, ASAE.metrics.E, ASAE_name, ASAE)
    print("Got bridge norms for ASAE")
    plot_bridge_mass_vs_eps(ASAE_bridge["bridge_gamma"], f"rzegzeg", f"bridge_mass_vs_eps_gamma", ASAE_name, ASAE.metrics.mu, ASAE.metrics.E, ASAE, weight_type="gamma", skip_plot=True)
    get_rho(ASAE)
    data_classifiability(ASAE, ASAE_name, eps=0.05, skip_plot=True)
    
    I_bi_A, T_bi_A, B_bi_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device)
    I_uni_A, T_uni_A, B_uni_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True)
    I_bridge_A, T_bridge_A, B_bridge_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=False, use_E_B=True)
    I_bridge_C_A, T_bridge_C_A, B_bridge_C_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True, use_E_B=True)
    print("Got bimodal data for ASAE")
    get_modality_gap(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    # get_modality_gap(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    # get_modality_gap(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    # get_modality_gap(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    get_contrastive_loss(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    get_contrastive_loss(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    get_contrastive_loss(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    get_contrastive_loss(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    print(ASAE.metrics)
    all_metrics.append(ASAE.metrics)
    all_saes.append(ASAE)
    bi_acc = ASAE.metrics.contrastive_loss_bimodal['accuracy']
    # uni_acc = ASAE.metrics.contrastive_loss_unimodal['accuracy']
    # bridge_acc = ASAE.metrics.contrastive_loss_bridge['accuracy']
    # bridge_C_acc = ASAE.metrics.contrastive_loss_bridge_C['accuracy']
    # print(f"Iteration {i+1} - Bimodal Acc: {bi_acc:.4f}, Unimodal Acc: {uni_acc:.4f}, Bridge Acc: {bridge_acc:.4f}, Bridge C Acc: {bridge_C_acc:.4f}")

print(all_metrics)
all_metrics_architecture = all_metrics
all_saes_architecture = all_saes
os.makedirs("./temporary_save", exist_ok=True)
torch.save(all_metrics_architecture, f"./temporary_save/all_metrics_architecture.pt")
torch.save(all_saes_architecture, f"./temporary_save/all_saes_architecture.pt")

In [ ]:
# TopK :

print(f"Training with TopK")
nb_concepts = 512 * 8
top_k = 20
d_model = 512

sae = overcomplete.sae.TopKSAE(d_model, nb_concepts=nb_concepts, top_k=top_k, device=device)
# copy_dict_weights = sae.dictionary._weights.clone().detach()
# sae.encoder.final_block[0].weight.data.copy_(copy_dict_weights)
alpha = 1e-3  # Dead features loss penalty

def _criterion(x, x_hat, pre_codes, codes, dictionary):
    # here we directly use the thresholds of the model to control the sparsity
    loss = (x - x_hat).square().mean()

    is_dead = ((codes > 0).sum(dim=0) == 0).float().detach()
    # we push the pre_codes (before relu) towards the positive orthant
    reanim_loss = (pre_codes * is_dead[None, :]).mean()

    loss -= reanim_loss * alpha

    return loss
criterion_1 = _criterion

sae.to(device)
sae.train()

optimizer = torch.optim.Adam(sae.parameters(), lr=5e-4)
criterion_2 = lambda *args, **kwargs: losses.alignment_penalty(*args, **kwargs, penalty=0, alignment_metric='cosim')
criterion = lambda *args, **kwargs: criterion_1(*args, **kwargs) + criterion_2(*args, **kwargs)

scheduler=None
steps_per_epoch = len(train_loader)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=5e-4, total_steps=5 * steps_per_epoch,
)

alpha_name = str(alpha).replace('.', '')
beta_name = str(0).replace('.', '')

CENTER_DATASET = False if 'CENTER_DATASET' not in locals() else CENTER_DATASET
sae_name = f"{DATASET}_{"TopK"}_centered_{CENTER_DATASET}_{8}_L0_{top_k}_alpha" + alpha_name + "beta" + beta_name + ".pt"
print(f"Model name: {sae_name}")

logs = train.train_multimodal_sae(
    sae, train_loader, criterion, optimizer, scheduler=scheduler, nb_epochs=5, device=device,
    monitoring=1, verbose=False,
    checkpoint_path="./checkpoints/",
    checkpoint_interval=5, checkpoint_name=sae_name,
)
ASAE = sae
ASAE_name = sae_name

ASAE.metrics = measure_everything(ASAE, train_loader, device, return_sqr=True)
# ASAE = best_
ASAE_bridge = {
    "bridge_sigma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False).cpu().detach(),
    "bridge_gamma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False, weight_type="OT").cpu().detach(),
}

get_sparse_reconstruction_metrics(ASAE, train_loader)
get_C_metrics(ASAE, train_loader, top_k=top_k)

# SSAE.metrics._add_more(get_stability_beta(train_loader, beta=0e-4), save=True)
# ASAE.metrics._add_more(get_stability_beta(train_loader, beta=5e-4), save=True)

get_D_structure_metrics(ASAE)
get_Z_structure_metrics(ASAE, train_loader)

get_bridge_norms(ASAE_bridge["bridge_sigma"], ASAE.metrics.mu, ASAE.metrics.E, ASAE_name, ASAE)
print("Got bridge norms for ASAE")
plot_bridge_mass_vs_eps(ASAE_bridge["bridge_gamma"], f"rzegzeg", f"bridge_mass_vs_eps_gamma", ASAE_name, ASAE.metrics.mu, ASAE.metrics.E, ASAE, weight_type="gamma", skip_plot=True)
get_rho(ASAE)
data_classifiability(ASAE, ASAE_name, eps=0.05, skip_plot=True)
    
I_bi_A, T_bi_A, B_bi_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device)
I_uni_A, T_uni_A, B_uni_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True)
I_bridge_A, T_bridge_A, B_bridge_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=False, use_E_B=True)
I_bridge_C_A, T_bridge_C_A, B_bridge_C_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True, use_E_B=True)
print("Got bimodal data for ASAE")
get_modality_gap(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
# get_modality_gap(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
# get_modality_gap(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
# get_modality_gap(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
get_contrastive_loss(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
get_contrastive_loss(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
get_contrastive_loss(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
get_contrastive_loss(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
print(ASAE.metrics)
all_metrics.append(ASAE.metrics)
all_saes.append(ASAE)
bi_acc = ASAE.metrics.contrastive_loss_bimodal['accuracy']
# uni_acc = ASAE.metrics.contrastive_loss_unimodal['accuracy']
# bridge_acc = ASAE.metrics.contrastive_loss_bridge['accuracy']
# bridge_C_acc = ASAE.metrics.contrastive_loss_bridge_C['accuracy']
# print(f"Iteration {i+1} - Bimodal Acc: {bi_acc:.4f}, Unimodal Acc: {uni_acc:.4f}, Bridge Acc: {bridge_acc:.4f}, Bridge C Acc: {bridge_C_acc:.4f}")

print(all_metrics)
all_metrics_architecture = all_metrics
all_saes_architecture = all_saes
os.makedirs("./temporary_save", exist_ok=True)
torch.save(all_metrics_architecture, f"./temporary_save/all_metrics_architecture.pt")
torch.save(all_saes_architecture, f"./temporary_save/all_saes_architecture.pt")

In [ ]:
# JumpReLU :

print(f"Training with JumpReLU")
nb_concepts = 512 * 8
top_k = 20
d_model = 512

def _criterion(x, x_hat, pre_codes, codes, dictionary):
    # here we directly use the thresholds of the model to control the sparsity
    loss = (x - x_hat).square().mean()

    sparsity = (codes > 0).float().mean().detach()
    if sparsity > desired_sparsity:
      # if we are not sparse enough, increase the thresholds levels
      loss -= sae.thresholds.sum()
      
    is_dead = ((codes > 0).sum(dim=0) == 0).float().detach()
    # we push the pre_codes (before relu) towards the positive orthant
    reanim_loss = (pre_codes * is_dead[None, :]).mean()

    loss -= reanim_loss * 1e-3

    return loss

sae = overcomplete.sae.JumpSAE(d_model, nb_concepts=nb_concepts, device=device)
alpha = 0.0
criterion_1 = lambda *args, **kwargs: _criterion(*args, **kwargs)

desired_sparsity = top_k / nb_concepts

sae.to(device)
sae.train()

optimizer = torch.optim.Adam(sae.parameters(), lr=5e-4)
criterion_2 = lambda *args, **kwargs: losses.alignment_penalty(*args, **kwargs, penalty=0, alignment_metric='cosim')
criterion = lambda *args, **kwargs: criterion_1(*args, **kwargs) + criterion_2(*args, **kwargs)

scheduler=None
steps_per_epoch = len(train_loader)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=5e-4, total_steps=5 * steps_per_epoch,
)

alpha_name = str(alpha).replace('.', '')
beta_name = str(0).replace('.', '')

CENTER_DATASET = False if 'CENTER_DATASET' not in locals() else CENTER_DATASET
sae_name = f"{DATASET}_{"JumpReLU"}_centered_{CENTER_DATASET}_{8}_L0_{top_k}_alpha" + alpha_name + "beta" + beta_name + ".pt"
print(f"Model name: {sae_name}")

logs = train.train_multimodal_sae(
    sae, train_loader, criterion, optimizer, scheduler=scheduler, nb_epochs=5, device=device,
    monitoring=1, verbose=False,
    checkpoint_path="./checkpoints/",
    checkpoint_interval=5, checkpoint_name=sae_name,
)
ASAE = sae
ASAE_name = sae_name

ASAE.metrics = measure_everything(ASAE, train_loader, device, return_sqr=True)
# ASAE = best_
ASAE_bridge = {
    "bridge_sigma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False).cpu().detach(),
    "bridge_gamma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False, weight_type="OT").cpu().detach(),
}

get_sparse_reconstruction_metrics(ASAE, train_loader)
get_C_metrics(ASAE, train_loader, top_k=50)

# SSAE.metrics._add_more(get_stability_beta(train_loader, beta=0e-4), save=True)
# ASAE.metrics._add_more(get_stability_beta(train_loader, beta=5e-4), save=True)

get_D_structure_metrics(ASAE)
get_Z_structure_metrics(ASAE, train_loader)

get_bridge_norms(ASAE_bridge["bridge_sigma"], ASAE.metrics.mu, ASAE.metrics.E, ASAE_name, ASAE)
print("Got bridge norms for ASAE")
plot_bridge_mass_vs_eps(ASAE_bridge["bridge_gamma"], f"rzegzeg", f"bridge_mass_vs_eps_gamma", ASAE_name, ASAE.metrics.mu, ASAE.metrics.E, ASAE, weight_type="gamma", skip_plot=True)
get_rho(ASAE)
data_classifiability(ASAE, ASAE_name, eps=0.05, skip_plot=True)

I_bi_A, T_bi_A, B_bi_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device)
I_uni_A, T_uni_A, B_uni_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True)
I_bridge_A, T_bridge_A, B_bridge_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=False, use_E_B=True)
I_bridge_C_A, T_bridge_C_A, B_bridge_C_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True, use_E_B=True)
print("Got bimodal data for ASAE")
get_modality_gap(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
# get_modality_gap(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
# get_modality_gap(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
# get_modality_gap(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
get_contrastive_loss(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
get_contrastive_loss(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
get_contrastive_loss(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
get_contrastive_loss(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
print(ASAE.metrics)
all_metrics.append(ASAE.metrics)
all_saes.append(ASAE)
bi_acc = ASAE.metrics.contrastive_loss_bimodal['accuracy']
# uni_acc = ASAE.metrics.contrastive_loss_unimodal['accuracy']
# bridge_acc = ASAE.metrics.contrastive_loss_bridge['accuracy']
# bridge_C_acc = ASAE.metrics.contrastive_loss_bridge_C['accuracy']
# print(f"Iteration {i+1} - Bimodal Acc: {bi_acc:.4f}, Unimodal Acc: {uni_acc:.4f}, Bridge Acc: {bridge_acc:.4f}, Bridge C Acc: {bridge_C_acc:.4f}")

print(all_metrics)
all_metrics_architecture = all_metrics
all_saes_architecture = all_saes
os.makedirs("./temporary_save", exist_ok=True)
torch.save(all_metrics_architecture, f"./temporary_save/all_metrics_architecture.pt")
torch.save(all_saes_architecture, f"./temporary_save/all_saes_architecture.pt")

In [ ]:
# 4e-4, L_align
best_ = None
best_bi_acc_ = 0
best_record = []
best_times = []
all_metrics = []
all_saes = []
i = 0
for _ in range(10):
    i += 1
    if i > 1:
        print(f"Iteration {i+1} - Training ASAE with beta=2e-4")
        print(f"Current best Bimodal Acc: {best_bi_acc_:.4f}. Corresponding Unimodal Acc: {best_.metrics.contrastive_loss_unimodal['accuracy']:.4f}, Bridge Acc: {best_.metrics.contrastive_loss_bridge['accuracy']:.4f}, Bridge C Acc: {best_.metrics.contrastive_loss_bridge_C['accuracy']:.4f}")
        print(f"Recorded best Bimodal Accs: {best_record}, at iterations: {best_times}")
    ASAE, ASAE_name = __train(beta=4e-4, force_retrain=True) # Aligned SAE
    ASAE.metrics = measure_everything(ASAE, train_loader, device, return_sqr=True)
    # ASAE = best_
    ASAE_bridge = {
        "bridge_sigma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False).cpu().detach(),
        "bridge_gamma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False, weight_type="OT").cpu().detach(),
    }
    get_bridge_norms(ASAE_bridge["bridge_sigma"], ASAE.metrics.mu, ASAE.metrics.E, ASAE_name, ASAE)
    print("Got bridge norms for ASAE")
    plot_bridge_mass_vs_eps(ASAE_bridge["bridge_gamma"], f"rzegzeg", f"bridge_mass_vs_eps_gamma", ASAE_name, ASAE.metrics.mu, ASAE.metrics.E, ASAE, weight_type="gamma", skip_plot=True)
    get_rho(ASAE)
    data_classifiability(ASAE, ASAE_name, eps=0.05, skip_plot=True)
    
    I_bi_A, T_bi_A, B_bi_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device)
    I_uni_A, T_uni_A, B_uni_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True)
    I_bridge_A, T_bridge_A, B_bridge_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=False, use_E_B=True)
    I_bridge_C_A, T_bridge_C_A, B_bridge_C_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True, use_E_B=True)
    print("Got bimodal data for ASAE")
    get_modality_gap(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    # get_modality_gap(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    # get_modality_gap(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    # get_modality_gap(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    get_contrastive_loss(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    get_contrastive_loss(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    get_contrastive_loss(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    get_contrastive_loss(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    print(ASAE.metrics)
    all_metrics.append(ASAE.metrics)
    all_saes.append(ASAE)
    bi_acc = ASAE.metrics.contrastive_loss_bimodal['accuracy']
    # uni_acc = ASAE.metrics.contrastive_loss_unimodal['accuracy']
    # bridge_acc = ASAE.metrics.contrastive_loss_bridge['accuracy']
    # bridge_C_acc = ASAE.metrics.contrastive_loss_bridge_C['accuracy']
    # print(f"Iteration {i+1} - Bimodal Acc: {bi_acc:.4f}, Unimodal Acc: {uni_acc:.4f}, Bridge Acc: {bridge_acc:.4f}, Bridge C Acc: {bridge_C_acc:.4f}")
    if bi_acc > best_bi_acc_:
        best_bi_acc_ = bi_acc
        best_ = ASAE
        print(f"New best model found with Bimodal Acc: {best_bi_acc_:.4f}")
        checkpoint = {
            'model_state_dict': best_.state_dict(),
            'logs': None
        }

        # check that checkpoint_path exists, otherwise create it
        checkpoint_path = f"./checkpoints/"
        checkpoint_name = ASAE_name.replace('.pt', '_best.pt')
        if not os.path.exists(checkpoint_path):
            os.makedirs(checkpoint_path)
        torch.save(checkpoint, f"{checkpoint_path}/{checkpoint_name}")
        best_record.append(best_bi_acc_)
        best_times.append(i)

print(all_metrics)
all_metrics_L_align = all_metrics
all_saes_L_align = all_saes
os.makedirs("./temporary_save", exist_ok=True)
torch.save(all_metrics_L_align, f"./temporary_save/all_metrics_L_align.pt")
torch.save(all_saes_L_align, f"./temporary_save/all_saes_L_align.pt")

In [ ]:
metrics_L_align = torch.load(f"./temporary_save/all_metrics_L_align.pt")
for i, m in enumerate(metrics_L_align):
    print(f"\nMetrics {i+1}:")
    print(f"% of E in bimodal: {m.E_in_B:.4f}")
    print(f"rho: {m.rho:.4f}")
    print(f"p_acc: {m.data_cassifiability["feature_probe"]:.3f}")
    print(f"delta_acc bimodal: {m.contrastive_loss_bimodal['accuracy']:.4f}")

In [ ]:
# 4e-4 L_align'

best_ = None
best_bi_acc_ = 0
best_record = []
best_times = []
all_metrics = []
all_saes = []
i = 0
for _ in range(10):
    i += 1
    if i > 1:
        print(f"Iteration {i+1} - Training ASAE with beta=2e-4")
        print(f"Current best Bimodal Acc: {best_bi_acc_:.4f}. Corresponding Unimodal Acc: {best_.metrics.contrastive_loss_unimodal['accuracy']:.4f}, Bridge Acc: {best_.metrics.contrastive_loss_bridge['accuracy']:.4f}, Bridge C Acc: {best_.metrics.contrastive_loss_bridge_C['accuracy']:.4f}")
        print(f"Recorded best Bimodal Accs: {best_record}, at iterations: {best_times}")
    ASAE, ASAE_name = __train(beta=4e-4, force_retrain=True, criterion_2_fct=losses.alignment_penalty_prime) # Aligned SAE
    ASAE.metrics = measure_everything(ASAE, train_loader, device, return_sqr=True)
    # ASAE = best_
    ASAE_bridge = {
        "bridge_sigma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False).cpu().detach(),
        "bridge_gamma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False, weight_type="OT").cpu().detach(),
    }
    get_bridge_norms(ASAE_bridge["bridge_sigma"], ASAE.metrics.mu, ASAE.metrics.E, ASAE_name, ASAE)
    print("Got bridge norms for ASAE")
    plot_bridge_mass_vs_eps(ASAE_bridge["bridge_gamma"], f"rzegzeg", f"bridge_mass_vs_eps_gamma", ASAE_name, ASAE.metrics.mu, ASAE.metrics.E, ASAE, weight_type="gamma", skip_plot=True)
    get_rho(ASAE)
    data_classifiability(ASAE, ASAE_name, eps=0.05, skip_plot=True)
    
    I_bi_A, T_bi_A, B_bi_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device)
    I_uni_A, T_uni_A, B_uni_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True)
    I_bridge_A, T_bridge_A, B_bridge_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=False, use_E_B=True)
    I_bridge_C_A, T_bridge_C_A, B_bridge_C_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True, use_E_B=True)
    print("Got bimodal data for ASAE")
    get_modality_gap(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    # get_modality_gap(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    # get_modality_gap(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    # get_modality_gap(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    get_contrastive_loss(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    get_contrastive_loss(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    get_contrastive_loss(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    get_contrastive_loss(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    print(ASAE.metrics)
    all_metrics.append(ASAE.metrics)
    all_saes.append(ASAE)
    bi_acc = ASAE.metrics.contrastive_loss_bimodal['accuracy']
    # uni_acc = ASAE.metrics.contrastive_loss_unimodal['accuracy']
    # bridge_acc = ASAE.metrics.contrastive_loss_bridge['accuracy']
    # bridge_C_acc = ASAE.metrics.contrastive_loss_bridge_C['accuracy']
    # print(f"Iteration {i+1} - Bimodal Acc: {bi_acc:.4f}, Unimodal Acc: {uni_acc:.4f}, Bridge Acc: {bridge_acc:.4f}, Bridge C Acc: {bridge_C_acc:.4f}")
    if bi_acc > best_bi_acc_:
        best_bi_acc_ = bi_acc
        best_ = ASAE
        print(f"New best model found with Bimodal Acc: {best_bi_acc_:.4f}")
        checkpoint = {
            'model_state_dict': best_.state_dict(),
            'logs': None
        }

        # check that checkpoint_path exists, otherwise create it
        checkpoint_path = f"./checkpoints/"
        checkpoint_name = ASAE_name.replace('.pt', '_best.pt')
        if not os.path.exists(checkpoint_path):
            os.makedirs(checkpoint_path)
        torch.save(checkpoint, f"{checkpoint_path}/{checkpoint_name}")
        best_record.append(best_bi_acc_)
        best_times.append(i)

print(all_metrics)
all_metrics_L_align_prime = all_metrics
all_saes_L_align_prime = all_saes
torch.save(all_metrics_L_align_prime, f"./temporary_save/all_metrics_L_align_prime.pt")
torch.save(all_saes_L_align_prime, f"./temporary_save/all_saes_L_align_prime.pt")

In [ ]:
metrics_L_align_prime = torch.load(f"./temporary_save/all_metrics_L_align_prime.pt")
for i, m in enumerate(metrics_L_align):
    print(f"\nMetrics {i+1}:")
    print(f"% of E in bimodal: {m.E_in_B:.4f}")
    print(f"rho: {m.rho:.4f}")
    print(f"p_acc: {m.data_cassifiability["feature_probe"]:.3f}")
    print(f"delta_acc bimodal: {m.contrastive_loss_bimodal['accuracy']:.4f}")

In [ ]:
best_ = None
best_bi_acc_ = 0
best_record = []
best_times = []
all_metrics = []
all_saes = []
i = 0
import importlib
import losses
importlib.reload(losses)
for _ in range(10):
    i += 1
    if i > 1:
        print(f"Iteration {i+1} - Training ASAE with beta=2e-4")
        print(f"Current best Bimodal Acc: {best_bi_acc_:.4f}. Corresponding Unimodal Acc: {best_.metrics.contrastive_loss_unimodal['accuracy']:.4f}, Bridge Acc: {best_.metrics.contrastive_loss_bridge['accuracy']:.4f}, Bridge C Acc: {best_.metrics.contrastive_loss_bridge_C['accuracy']:.4f}")
        print(f"Recorded best Bimodal Accs: {best_record}, at iterations: {best_times}")
    beta = 2e-4
    ASAE, ASAE_name = __train(beta=beta, force_retrain=True, criterion_2_fct=losses.alignment_penalty_distribution) # Aligned SAE
    ASAE.metrics = measure_everything(ASAE, train_loader, device, return_sqr=True)
    print(f"ASAE metrics: {ASAE.metrics}")
    # ASAE = best_
    ASAE_bridge = {
        "bridge_sigma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False).cpu().detach(),
        "bridge_gamma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False, weight_type="OT").cpu().detach(),
    }
    get_bridge_norms(ASAE_bridge["bridge_sigma"], ASAE.metrics.mu, ASAE.metrics.E, ASAE_name, ASAE)
    print("Got bridge norms for ASAE")
    plot_bridge_mass_vs_eps(ASAE_bridge["bridge_gamma"], f"rzegzeg", f"bridge_mass_vs_eps_gamma", ASAE_name, ASAE.metrics.mu, ASAE.metrics.E, ASAE, weight_type="gamma", skip_plot=True)
    get_rho(ASAE)
    data_classifiability(ASAE, ASAE_name, eps=0.05, skip_plot=True)
    
    I_bi_A, T_bi_A, B_bi_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device)
    I_uni_A, T_uni_A, B_uni_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True)
    I_bridge_A, T_bridge_A, B_bridge_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=False, use_E_B=True)
    I_bridge_C_A, T_bridge_C_A, B_bridge_C_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True, use_E_B=True)
    print("Got bimodal data for ASAE")
    get_modality_gap(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    # get_modality_gap(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    # get_modality_gap(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    # get_modality_gap(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    get_contrastive_loss(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    get_contrastive_loss(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    get_contrastive_loss(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    get_contrastive_loss(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    print(ASAE.metrics)
    all_metrics.append(ASAE.metrics)
    all_saes.append(ASAE)
    bi_acc = ASAE.metrics.contrastive_loss_bimodal['accuracy']
    # uni_acc = ASAE.metrics.contrastive_loss_unimodal['accuracy']
    # bridge_acc = ASAE.metrics.contrastive_loss_bridge['accuracy']
    # bridge_C_acc = ASAE.metrics.contrastive_loss_bridge_C['accuracy']
    # print(f"Iteration {i+1} - Bimodal Acc: {bi_acc:.4f}, Unimodal Acc: {uni_acc:.4f}, Bridge Acc: {bridge_acc:.4f}, Bridge C Acc: {bridge_C_acc:.4f}")
    if bi_acc > best_bi_acc_:
        best_bi_acc_ = bi_acc
        best_ = ASAE
        print(f"New best model found with Bimodal Acc: {best_bi_acc_:.4f}")
        checkpoint = {
            'model_state_dict': best_.state_dict(),
            'logs': None
        }

        # check that checkpoint_path exists, otherwise create it
        checkpoint_path = f"./checkpoints/"
        checkpoint_name = ASAE_name.replace('.pt', '_best.pt')
        if not os.path.exists(checkpoint_path):
            os.makedirs(checkpoint_path)
        torch.save(checkpoint, f"{checkpoint_path}/{checkpoint_name}")
        best_record.append(best_bi_acc_)
        best_times.append(i)

print(all_metrics)
all_metrics_L_align_dist = all_metrics
all_saes_L_align_dist = all_saes
torch.save(all_metrics_L_align_dist, f"./temporary_save/all_metrics_L_align_dist.pt")
torch.save(all_saes_L_align_dist, f"./temporary_save/all_saes_L_align_dist.pt")

In [ ]:
best_ = None
best_bi_acc_ = 0
best_record = []
best_times = []
all_metrics = []
all_saes = []
i = 0
import importlib
import losses
losses = importlib.reload(losses)
for _ in range(10):
    i += 1
    if i > 1:
        print(f"Iteration {i+1} - Training ASAE with beta=2e-4")
        print(f"Current best Bimodal Acc: {best_bi_acc_:.4f}. Corresponding Unimodal Acc: {best_.metrics.contrastive_loss_unimodal['accuracy']:.4f}, Bridge Acc: {best_.metrics.contrastive_loss_bridge['accuracy']:.4f}, Bridge C Acc: {best_.metrics.contrastive_loss_bridge_C['accuracy']:.4f}")
        print(f"Recorded best Bimodal Accs: {best_record}, at iterations: {best_times}")
    ASAE, ASAE_name = __train(beta=4e-4, force_retrain=True, criterion_2_fct=losses.alignment_penalty_distribution_prime) # Aligned SAE
    ASAE.metrics = measure_everything(ASAE, train_loader, device, return_sqr=True)
    # ASAE = best_
    ASAE_bridge = {
        "bridge_sigma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False).cpu().detach(),
        "bridge_gamma" : measure_bridge_sigma(ASAE, train_loader, ASAE.metrics.E, ASAE.metrics.E_Img, ASAE.metrics.E_Txt, device, center=False, normalize=False, null_C=False, null_D=False, weight_type="OT").cpu().detach(),
    }
    get_bridge_norms(ASAE_bridge["bridge_sigma"], ASAE.metrics.mu, ASAE.metrics.E, ASAE_name, ASAE)
    print("Got bridge norms for ASAE")
    plot_bridge_mass_vs_eps(ASAE_bridge["bridge_gamma"], f"rzegzeg", f"bridge_mass_vs_eps_gamma", ASAE_name, ASAE.metrics.mu, ASAE.metrics.E, ASAE, weight_type="gamma", skip_plot=True)
    get_rho(ASAE)
    data_classifiability(ASAE, ASAE_name, eps=0.05, skip_plot=True)
    
    I_bi_A, T_bi_A, B_bi_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device)
    I_uni_A, T_uni_A, B_uni_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True)
    I_bridge_A, T_bridge_A, B_bridge_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=False, use_E_B=True)
    I_bridge_C_A, T_bridge_C_A, B_bridge_C_A = get_bimodal_data(ASAE, train_loader, ASAE.metrics.mu, device, complement=True, use_E_B=True)
    print("Got bimodal data for ASAE")
    get_modality_gap(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    # get_modality_gap(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    # get_modality_gap(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    # get_modality_gap(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    get_contrastive_loss(I_bi_A, T_bi_A, ASAE, data_name="bimodal")
    get_contrastive_loss(I_uni_A, T_uni_A, ASAE, data_name="unimodal")
    get_contrastive_loss(I_bridge_A, T_bridge_A, ASAE, data_name="bridge")
    get_contrastive_loss(I_bridge_C_A, T_bridge_C_A, ASAE, data_name="bridge_C")
    print(ASAE.metrics)
    all_metrics.append(ASAE.metrics)
    all_saes.append(ASAE)
    bi_acc = ASAE.metrics.contrastive_loss_bimodal['accuracy']
    # uni_acc = ASAE.metrics.contrastive_loss_unimodal['accuracy']
    # bridge_acc = ASAE.metrics.contrastive_loss_bridge['accuracy']
    # bridge_C_acc = ASAE.metrics.contrastive_loss_bridge_C['accuracy']
    # print(f"Iteration {i+1} - Bimodal Acc: {bi_acc:.4f}, Unimodal Acc: {uni_acc:.4f}, Bridge Acc: {bridge_acc:.4f}, Bridge C Acc: {bridge_C_acc:.4f}")
    if bi_acc > best_bi_acc_:
        best_bi_acc_ = bi_acc
        best_ = ASAE
        print(f"New best model found with Bimodal Acc: {best_bi_acc_:.4f}")
        checkpoint = {
            'model_state_dict': best_.state_dict(),
            'logs': None
        }

        # check that checkpoint_path exists, otherwise create it
        checkpoint_path = f"./checkpoints/"
        checkpoint_name = ASAE_name.replace('.pt', '_best.pt')
        if not os.path.exists(checkpoint_path):
            os.makedirs(checkpoint_path)
        torch.save(checkpoint, f"{checkpoint_path}/{checkpoint_name}")
        best_record.append(best_bi_acc_)
        best_times.append(i)

print(all_metrics)
all_metrics_L_align_dist = all_metrics
all_saes_L_align_dist = all_saes
torch.save(all_metrics_L_align_dist, f"./temporary_save/all_metrics_L_align_dist.pt")
torch.save(all_saes_L_align_dist, f"./temporary_save/all_saes_L_align_dist.pt")

- rename delta_acc to Delta recall. Rename acc to recall everywhere. Use Zero shot acc, never acc. Only use acc for feature as probes setups.

- 3D LR projection : say "we find that concepts are neatly organized in latent space, clustered by modality".

- More models ! OpenClip, eva clip, clipa, siglip, various sizes, try them all !
    - measure recall@k on unmodified embeddings to check if it's consistent with literature so if our embeddings are properly extracted.
    - do the shit anyway